第一波特征：lr 0.71。categorical feature 做target、onehot、label encoding，加上normal features，60维

第二波特征：lr 0.75。对连续型，数值型变量做离散化处理。998维

第三波特征：lr 0.76。引入bureau特征。1468维

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import re
import category_encoders as ce
from sklearn.preprocessing import LabelBinarizer, StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2, SelectFromModel, mutual_info_classif
from sklearn.ensemble import ExtraTreesClassifier

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
train = pd.read_csv('./data/rawdata/application_train.csv.zip',compression='zip')
test = pd.read_csv('./data/rawdata/application_test.csv.zip',compression='zip')

In [3]:
train.shape
train.head()
train.tail()
test.shape
test.tail()

(307511, 122)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
307510,456255,0,Cash loans,F,N,N,0,157500.0,675000.0,49117.5,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,1.0


(48744, 121)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,3.0,1.0
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
48743,456250,Cash loans,F,Y,N,0,135000.0,312768.0,24709.5,270000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0


In [4]:
#  add features from other source
bureau_features = pd.read_csv('./data/rawdata/bureau_feature.csv')
bureau_features.columns = ['bureau_' + re.sub(',|:| ','_',c) if c != 'SK_ID_CURR' else c for c in bureau_features.columns]
bureau_features.shape

train = train.merge(bureau_features,how='left',on='SK_ID_CURR')
test = test.merge(bureau_features,how='left',on='SK_ID_CURR')
del bureau_features

In [5]:
train.shape
train.head()
train.tail()
test.shape
test.tail()

(305811, 31)

(307511, 152)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,bureau_CREDIT_CURRENCY_cnt,bureau_CREDIT_ACTIVE_cnt,bureau_CREDIT_TYPE_cnt,bureau_STATUS_set_cnt,bureau_latest_STATUS_cnt,bureau_CREDIT_CURRENCY_set,bureau_CREDIT_ACTIVE_set,bureau_CREDIT_TYPE_set,bureau_latest_STATUS_set,bureau_STATUS_set
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,1.0,2.0,2.0,4.0,2.0,{'currency 1'},"{'Closed', 'Active'}","{'Credit card', 'Consumer credit'}","{'C', '0'}","X,C,0,1"
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,1.0,2.0,2.0,0.0,0.0,{'currency 1'},"{'Closed', 'Active'}","{'Credit card', 'Consumer credit'}",{nan},NaN
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,1.0,1.0,1.0,0.0,0.0,{'currency 1'},{'Closed'},{'Consumer credit'},{nan},NaN
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,1.0,1.0,1.0,0.0,0.0,{'currency 1'},{'Closed'},{'Consumer credit'},{nan},NaN


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,bureau_CREDIT_CURRENCY_cnt,bureau_CREDIT_ACTIVE_cnt,bureau_CREDIT_TYPE_cnt,bureau_STATUS_set_cnt,bureau_latest_STATUS_cnt,bureau_CREDIT_CURRENCY_set,bureau_CREDIT_ACTIVE_set,bureau_CREDIT_TYPE_set,bureau_latest_STATUS_set,bureau_STATUS_set
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,1.0,2.0,2.0,2.0,2.0,{'currency 1'},"{'Closed', 'Active'}","{'Credit card', 'Consumer credit'}","{'X', 'C'}","X,C,0"
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,1.0,1.0,1.0,1.0,1.0,{'currency 1'},{'Closed'},{'Consumer credit'},{'C'},"C,0"
307510,456255,0,Cash loans,F,N,N,0,157500.0,675000.0,49117.5,...,1.0,2.0,2.0,1.0,1.0,{'currency 1'},"{'Closed', 'Active'}","{'Credit card', 'Consumer credit'}",{'C'},"X,C,0"


(48744, 151)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,bureau_CREDIT_CURRENCY_cnt,bureau_CREDIT_ACTIVE_cnt,bureau_CREDIT_TYPE_cnt,bureau_STATUS_set_cnt,bureau_latest_STATUS_cnt,bureau_CREDIT_CURRENCY_set,bureau_CREDIT_ACTIVE_set,bureau_CREDIT_TYPE_set,bureau_latest_STATUS_set,bureau_STATUS_set
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,1.0,2.0,3.0,4.0,3.0,{'currency 1'},"{'Closed', 'Active'}","{'Credit card', 'Microloan', 'Consumer credit'}","{'X', 'C', '0'}","X,C,0,1"
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,1.0,2.0,3.0,4.0,2.0,{'currency 1'},"{'Closed', 'Active'}","{'Credit card', 'Mortgage', 'Consumer credit'}","{'X', 'C'}","X,C,0,1"
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,1.0,2.0,2.0,6.0,3.0,{'currency 1'},"{'Closed', 'Active'}","{'Credit card', 'Consumer credit'}","{'X', 'C', '0'}","X,1,C,2,5,0,4,3"
48743,456250,Cash loans,F,Y,N,0,135000.0,312768.0,24709.5,270000.0,...,1.0,2.0,2.0,2.0,3.0,{'currency 1'},"{'Closed', 'Active'}","{'Credit card', 'Consumer credit'}","{'X', 'C', '0'}","X,C,0"


In [6]:
#  add features from other source
cash_features = pd.read_csv('./data/rawdata/cash_features.csv')
cash_features.columns = ['cash_' + re.sub(',|:| ','_',c) if c != 'SK_ID_CURR' else c for c in cash_features.columns]
cash_features.shape
cash_features.columns

train = train.merge(cash_features,how='left',on='SK_ID_CURR')
test = test.merge(cash_features,how='left',on='SK_ID_CURR')

del cash_features

In [7]:
train.shape
train.head()
train.tail()
test.shape
test.tail()

(337252, 18)

Index(['SK_ID_CURR', 'cash_MONTHS_BALANCE_min', 'cash_MONTHS_BALANCE_max',
       'cash_MONTHS_BALANCE_cnt', 'cash_CNT_INSTALMENT_min',
       'cash_CNT_INSTALMENT_max', 'cash_CNT_INSTALMENT_set',
       'cash_CNT_INSTALMENT_FUTURE_min', 'cash_CNT_INSTALMENT_FUTURE_max',
       'cash_NAME_CONTRACT_STATUS_cntd', 'cash_NAME_CONTRACT_STATUS_set',
       'cash_NAME_CONTRACT_STATUS_latest', 'cash_SK_DPD_max',
       'cash_SK_DPD_cnt0', 'cash_SK_DPD_DEF_max', 'cash_SK_DPD_DEF_cnt0',
       'cash_SK_DPD_diff_max', 'cash_SK_DPD_diff_cnt0'],
      dtype='object')

(307511, 169)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,cash_CNT_INSTALMENT_FUTURE_max,cash_NAME_CONTRACT_STATUS_cntd,cash_NAME_CONTRACT_STATUS_set,cash_NAME_CONTRACT_STATUS_latest,cash_SK_DPD_max,cash_SK_DPD_cnt0,cash_SK_DPD_DEF_max,cash_SK_DPD_DEF_cnt0,cash_SK_DPD_diff_max,cash_SK_DPD_diff_cnt0
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,24.0,1.0,Active,Active,0.0,0.0,0.0,0.0,0.0,0.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,12.0,2.0,"Completed,Active","Completed,Active",0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,4.0,2.0,"Completed,Active",NaN,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,48.0,3.0,"Returned to the store,Completed,Active",NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,24.0,3.0,"Signed,Completed,Active","Completed,Active",0.0,0.0,0.0,0.0,0.0,0.0


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,cash_CNT_INSTALMENT_FUTURE_max,cash_NAME_CONTRACT_STATUS_cntd,cash_NAME_CONTRACT_STATUS_set,cash_NAME_CONTRACT_STATUS_latest,cash_SK_DPD_max,cash_SK_DPD_cnt0,cash_SK_DPD_DEF_max,cash_SK_DPD_DEF_cnt0,cash_SK_DPD_diff_max,cash_SK_DPD_diff_cnt0
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,8.0,3.0,"Signed,Completed,Active",NaN,0.0,0.0,0.0,0.0,0.0,0.0
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,6.0,2.0,"Completed,Active",NaN,0.0,0.0,0.0,0.0,0.0,0.0
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,6.0,2.0,"Completed,Active",Completed,5.0,1.0,5.0,1.0,0.0,0.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,16.0,1.0,Active,NaN,0.0,0.0,0.0,0.0,0.0,0.0
307510,456255,0,Cash loans,F,N,N,0,157500.0,675000.0,49117.5,...,36.0,2.0,"Completed,Active","Completed,Active",5.0,1.0,5.0,1.0,0.0,0.0


(48744, 168)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,cash_CNT_INSTALMENT_FUTURE_max,cash_NAME_CONTRACT_STATUS_cntd,cash_NAME_CONTRACT_STATUS_set,cash_NAME_CONTRACT_STATUS_latest,cash_SK_DPD_max,cash_SK_DPD_cnt0,cash_SK_DPD_DEF_max,cash_SK_DPD_DEF_cnt0,cash_SK_DPD_diff_max,cash_SK_DPD_diff_cnt0
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,24.0,2.0,"Completed,Active",NaN,0.0,0.0,0.0,0.0,0.0,0.0
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,24.0,3.0,"Signed,Completed,Active","Completed,Active",0.0,0.0,0.0,0.0,0.0,0.0
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,12.0,2.0,"Completed,Active",Completed,0.0,0.0,0.0,0.0,0.0,0.0
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,24.0,3.0,"Signed,Completed,Active",NaN,0.0,0.0,0.0,0.0,0.0,0.0
48743,456250,Cash loans,F,Y,N,0,135000.0,312768.0,24709.5,270000.0,...,12.0,2.0,"Completed,Active","Completed,Active",0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
#  add features from other source
credit_features = pd.read_csv('./data/rawdata/credit_features.csv')
credit_features.columns = ['credit_' + re.sub(',|:| ','_',c) if c != 'SK_ID_CURR' else c for c in credit_features.columns]
credit_features.shape
credit_features.columns

train = train.merge(credit_features,how='left',on='SK_ID_CURR')
test = test.merge(credit_features,how='left',on='SK_ID_CURR')

del credit_features

In [9]:
train.shape
train.head()
train.tail()
test.shape
test.tail()

(103558, 72)

Index(['SK_ID_CURR', 'credit_MONTHS_BALANCE_min', 'credit_MONTHS_BALANCE_max',
       'credit_MONTHS_BALANCE_count', 'credit_AMT_BALANCE_min',
       'credit_AMT_BALANCE_max', 'credit_AMT_BALANCE_sum',
       'credit_AMT_BALANCE_mean', 'credit_AMT_BALANCE_size',
       'credit_AMT_CREDIT_LIMIT_ACTUAL_min',
       'credit_AMT_CREDIT_LIMIT_ACTUAL_max',
       'credit_AMT_DRAWINGS_ATM_CURRENT_min',
       'credit_AMT_DRAWINGS_ATM_CURRENT_size',
       'credit_AMT_DRAWINGS_ATM_CURRENT_mean',
       'credit_AMT_DRAWINGS_ATM_CURRENT_sum',
       'credit_AMT_DRAWINGS_ATM_CURRENT_max',
       'credit_AMT_DRAWINGS_CURRENT_min', 'credit_AMT_DRAWINGS_CURRENT_size',
       'credit_AMT_DRAWINGS_CURRENT_mean', 'credit_AMT_DRAWINGS_CURRENT_sum',
       'credit_AMT_DRAWINGS_CURRENT_max',
       'credit_AMT_DRAWINGS_OTHER_CURRENT_min',
       'credit_AMT_DRAWINGS_OTHER_CURRENT_size',
       'credit_AMT_DRAWINGS_OTHER_CURRENT_mean',
       'credit_AMT_DRAWINGS_OTHER_CURRENT_sum',
       'credit_AMT_DRAW

(307511, 240)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,credit_CNT_INSTALMENT_MATURE_CUM_set,credit_CNT_INSTALMENT_MATURE_CUM_latest,credit_NAME_CONTRACT_STATUS_cntd,credit_NAME_CONTRACT_STATUS_set,credit_SK_DPD_max,credit_SK_DPD_size,credit_SK_DPD_DEF_max,credit_SK_DPD_DEF_size,credit_SK_DPD_diff_max,credit_SK_DPD_diff_size
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0.0,0.0,1.0,Active,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,credit_CNT_INSTALMENT_MATURE_CUM_set,credit_CNT_INSTALMENT_MATURE_CUM_latest,credit_NAME_CONTRACT_STATUS_cntd,credit_NAME_CONTRACT_STATUS_set,credit_SK_DPD_max,credit_SK_DPD_size,credit_SK_DPD_DEF_max,credit_SK_DPD_DEF_size,credit_SK_DPD_diff_max,credit_SK_DPD_diff_size
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307510,456255,0,Cash loans,F,N,N,0,157500.0,675000.0,49117.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(48744, 239)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,credit_CNT_INSTALMENT_MATURE_CUM_set,credit_CNT_INSTALMENT_MATURE_CUM_latest,credit_NAME_CONTRACT_STATUS_cntd,credit_NAME_CONTRACT_STATUS_set,credit_SK_DPD_max,credit_SK_DPD_size,credit_SK_DPD_DEF_max,credit_SK_DPD_DEF_size,credit_SK_DPD_diff_max,credit_SK_DPD_diff_size
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48743,456250,Cash loans,F,Y,N,0,135000.0,312768.0,24709.5,270000.0,...,"4.0,9.0,5.0,1.0,8.0,7.0,6.0,3.0,2.0,0.0,10.0",10.0,1.0,Active,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
#  add features from other source
installment_features = pd.read_csv('./data/rawdata/installment_features.csv')
installment_features.columns = ['installment_' + re.sub(',|:| ','_',c) if c != 'SK_ID_CURR' else c for c in installment_features.columns]
installment_features.shape
installment_features.columns

train = train.merge(installment_features,how='left',on='SK_ID_CURR')
test = test.merge(installment_features,how='left',on='SK_ID_CURR')

del installment_features

In [11]:

train.shape
train.head()
train.tail()
test.shape
test.tail()

(339587, 45)

Index(['SK_ID_CURR', 'installment_NUM_INSTALMENT_VERSION_nunique',
       'installment_NUM_INSTALMENT_VERSION_set',
       'installment_NUM_INSTALMENT_VERSION_latest',
       'installment_NUM_INSTALMENT_NUMBER_min',
       'installment_NUM_INSTALMENT_NUMBER_max',
       'installment_NUM_INSTALMENT_NUMBER_count',
       'installment_NUM_INSTALMENT_NUMBER_nunique',
       'installment_DAYS_INSTALMENT_min', 'installment_DAYS_INSTALMENT_max',
       'installment_DAYS_INSTALMENT_median',
       'installment_DAYS_INSTALMENT_mean', 'installment_DAYS_INSTALMENT_range',
       'installment_DAYS_ENTRY_PAYMENT_min',
       'installment_DAYS_ENTRY_PAYMENT_max',
       'installment_DAYS_ENTRY_PAYMENT_median',
       'installment_DAYS_ENTRY_PAYMENT_mean',
       'installment_DAYS_ENTRY_PAYMENT_range', 'installment_DPD_min',
       'installment_DPD_max', 'installment_DPD_median', 'installment_DPD_mean',
       'installment_DPD_sum', 'installment_DPD_std',
       'installment_AMT_INSTALMENT_min', 'ins

(307511, 284)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,installment_AMT_PAYMENT_median,installment_AMT_PAYMENT_std,installment_AMT_PAYMENT_nunique,installment_AMT_DPD_min,installment_AMT_DPD_max,installment_AMT_DPD_mean,installment_AMT_DPD_sum,installment_AMT_DPD_median,installment_AMT_DPD_std,installment_AMT_DPD_nunique
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,9251.775,10058.037722,2.0,0.0,0.000,0.000000,0.000,0.0,0.000000,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,64275.615,110542.592300,6.0,0.0,0.000,0.000000,0.000,0.0,0.000000,1.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,5357.250,3011.871810,2.0,0.0,0.000,0.000000,0.000,0.0,0.000000,1.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,29027.520,168097.624347,4.0,0.0,0.000,0.000000,0.000,0.0,0.000000,1.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,16037.640,8048.060877,14.0,0.0,22655.655,452.384318,29857.365,0.0,2843.383508,7.0


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,installment_AMT_PAYMENT_median,installment_AMT_PAYMENT_std,installment_AMT_PAYMENT_nunique,installment_AMT_DPD_min,installment_AMT_DPD_max,installment_AMT_DPD_mean,installment_AMT_DPD_sum,installment_AMT_DPD_median,installment_AMT_DPD_std,installment_AMT_DPD_nunique
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,6605.910,2346.819209,2.0,0.000,0.000,0.00000,0.000,0.0,0.000000,1.0
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,10074.465,11.261529,2.0,0.000,0.000,0.00000,0.000,0.0,0.000000,1.0
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,3973.095,1638.228743,8.0,0.000,3945.825,283.79250,3973.095,0.0,1054.030507,3.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,2296.440,8602.512994,2.0,0.000,0.000,0.00000,0.000,0.0,0.000000,1.0
307510,456255,0,Cash loans,F,N,N,0,157500.0,675000.0,49117.5,...,27489.690,115619.974324,30.0,-615229.515,28641.150,-6181.50223,-457431.165,0.0,72500.592363,21.0


(48744, 283)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,installment_AMT_PAYMENT_median,installment_AMT_PAYMENT_std,installment_AMT_PAYMENT_nunique,installment_AMT_DPD_min,installment_AMT_DPD_max,installment_AMT_DPD_mean,installment_AMT_DPD_sum,installment_AMT_DPD_median,installment_AMT_DPD_std,installment_AMT_DPD_nunique
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,14222.4300,133045.794034,2.0,0.0,0.00,0.000000,0.000,0.0,0.000000,1.0
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,5448.6450,4783.524039,12.0,0.0,9680.49,314.714589,22974.165,0.0,1422.346320,5.0
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,15761.2050,23487.981155,3.0,0.0,0.00,0.000000,0.000,0.0,0.000000,1.0
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,23451.7050,8672.791892,3.0,0.0,0.00,0.000000,0.000,0.0,0.000000,1.0
48743,456250,Cash loans,F,Y,N,0,135000.0,312768.0,24709.5,270000.0,...,8724.0825,10518.273588,28.0,0.0,9000.00,193.079700,9653.985,0.0,1274.264156,3.0


In [12]:
#  add features from other source
prev_features = pd.read_csv('./data/rawdata/prev_features.csv')
prev_features.columns = ['prev_' + re.sub(',|:| ','_',c)  if c != 'SK_ID_CURR' else c for c in prev_features.columns]
prev_features.shape
prev_features.columns

train = train.merge(prev_features,how='left',on='SK_ID_CURR')
test = test.merge(prev_features,how='left',on='SK_ID_CURR')

del prev_features

In [13]:
train.shape
train.head()
train.tail()
test.shape
test.tail()

(338857, 842)

Index(['SK_ID_CURR', 'prev_PREV_cnt', 'prev_AMT_DOWN_PAYMENT_cnt_negative',
       'prev_AMT_DOWN_PAYMENT_cnt_positive',
       'prev_HOUR_APPR_PROCESS_START_median',
       'prev_FLAG_LAST_APPL_PER_CONTRACT_1', 'prev_NFLAG_LAST_APPL_IN_DAY_1',
       'prev_NFLAG_INSURED_ON_APPROVAL', 'prev_NAME_CONTRACT_STATUS_Approved',
       'prev_NAME_CONTRACT_STATUS_Canceled',
       ...
       'prev_AMT_CREDIT_max', 'prev_AMT_CREDIT_min',
       'prev_AMT_GOODS_PRICE_sum', 'prev_AMT_GOODS_PRICE_mean',
       'prev_AMT_GOODS_PRICE_max', 'prev_AMT_GOODS_PRICE_min',
       'prev_AMT_DOWN_PAYMENT_sum', 'prev_AMT_DOWN_PAYMENT_mean',
       'prev_AMT_DOWN_PAYMENT_max', 'prev_AMT_DOWN_PAYMENT_min'],
      dtype='object', length=842)

(307511, 1125)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,prev_AMT_CREDIT_max,prev_AMT_CREDIT_min,prev_AMT_GOODS_PRICE_sum,prev_AMT_GOODS_PRICE_mean,prev_AMT_GOODS_PRICE_max,prev_AMT_GOODS_PRICE_min,prev_AMT_DOWN_PAYMENT_sum,prev_AMT_DOWN_PAYMENT_mean,prev_AMT_DOWN_PAYMENT_max,prev_AMT_DOWN_PAYMENT_min
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,179055.0,179055.0,179055.00,179055.00,179055.0,179055.00,0.00,0.00,0.0,0.00
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,1035882.0,68053.5,1306309.50,435436.50,900000.0,68809.50,6885.00,3442.50,6885.0,0.00
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,20106.0,20106.0,24282.00,24282.00,24282.0,24282.00,4860.00,4860.00,4860.0,4860.00
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,906615.0,0.0,2449829.34,408304.89,688500.0,26912.34,69680.34,34840.17,66987.0,2693.34
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,284400.0,14616.0,903181.50,150530.25,247500.0,17176.50,6781.50,3390.75,3676.5,3105.00


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,prev_AMT_CREDIT_max,prev_AMT_CREDIT_min,prev_AMT_GOODS_PRICE_sum,prev_AMT_GOODS_PRICE_mean,prev_AMT_GOODS_PRICE_max,prev_AMT_GOODS_PRICE_min,prev_AMT_DOWN_PAYMENT_sum,prev_AMT_DOWN_PAYMENT_mean,prev_AMT_DOWN_PAYMENT_max,prev_AMT_DOWN_PAYMENT_min
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,40455.0,40455.0,40455.0,40455.000,40455.0,40455.0,0.0,0.00,0.0,0.0
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,56821.5,56821.5,57595.5,57595.500,57595.5,57595.5,3456.0,3456.00,3456.0,3456.0
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,27306.0,13945.5,48325.5,24162.750,28912.5,19413.0,8806.5,4403.25,5913.0,2893.5
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,247423.5,21456.0,242635.5,121317.750,223789.5,18846.0,0.0,0.00,0.0,0.0
307510,456255,0,Cash loans,F,N,N,0,157500.0,675000.0,49117.5,...,1271929.5,45000.0,2902167.0,362770.875,1170000.0,45000.0,14823.0,4941.00,9000.0,0.0


(48744, 1124)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,prev_AMT_CREDIT_max,prev_AMT_CREDIT_min,prev_AMT_GOODS_PRICE_sum,prev_AMT_GOODS_PRICE_mean,prev_AMT_GOODS_PRICE_max,prev_AMT_GOODS_PRICE_min,prev_AMT_DOWN_PAYMENT_sum,prev_AMT_DOWN_PAYMENT_mean,prev_AMT_DOWN_PAYMENT_max,prev_AMT_DOWN_PAYMENT_min
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,254700.0,254700.000,225000.000,225000.0000,225000.0,225000.000,NaN,NaN,NaN,NaN
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,208854.0,26788.500,347485.500,86871.3750,184500.0,28345.500,3600.0,1200.00,3600.0,0.0
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,153738.0,111295.665,282120.165,141060.0825,170824.5,111295.665,17086.5,8543.25,17086.5,0.0
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,453874.5,0.000,568791.000,142197.7500,400950.0,41886.000,4500.0,1500.00,4500.0,0.0
48743,456250,Cash loans,F,Y,N,0,135000.0,312768.0,24709.5,270000.0,...,533160.0,0.000,1062661.500,212532.3000,450000.0,66091.500,0.0,0.00,0.0,0.0


In [14]:
def duplicate_columns(df, return_dataframe = False, verbose = False):
    '''
        a function to detect and possibly remove duplicated columns for a pandas dataframe
    '''
    from pandas.core.common import array_equivalent
    # group columns by dtypes, only the columns of the same dtypes can be duplicate of each other
    groups = df.columns.to_series().groupby(df.dtypes).groups
    duplicated_columns = []

    for dtype, col_names in groups.items():
        column_values = df[col_names]
        num_columns = len(col_names)

        # find duplicated columns by checking pairs of columns, store first column name if duplicate exist 
        for i in range(num_columns):
            column_i = column_values.iloc[:,i].values
            for j in range(i + 1, num_columns):
                column_j = column_values.iloc[:,j].values
                if array_equivalent(column_i, column_j):
                    if verbose: 
                        print("column {} is a duplicate of column {}".format(col_names[i], col_names[j]))
                    duplicated_columns.append(col_names[i])
                    break
    if not return_dataframe:
        # return the column names of those duplicated exists
        return duplicated_columns
    else:
        # return a dataframe with duplicated columns dropped 
        return df.drop(labels = duplicated_columns, axis = 1)

In [15]:
train_s = duplicate_columns(train, return_dataframe=True,verbose = True)

/Users/finup/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: 'pandas.core.common.array_equivalent' is deprecated and is no longer public API


column prev_NAME_CONTRACT_STATUS_Unused_offer is a duplicate of column prev_CODE_REJECT_REASON_CLIENT
column prev_credit_MONTHS_BALANCE_max_min is a duplicate of column prev_credit_MONTHS_BALANCE_max_max
column prev_credit_MONTHS_BALANCE_max_max is a duplicate of column prev_credit_MONTHS_BALANCE_max_mean
column prev_credit_MONTHS_BALANCE_max_mean is a duplicate of column prev_credit_MONTHS_BALANCE_max_median
column prev_credit_AMT_BALANCE_min_mean is a duplicate of column prev_credit_AMT_BALANCE_min_median
column prev_credit_AMT_BALANCE_mean_mean is a duplicate of column prev_credit_AMT_BALANCE_mean_median
column prev_credit_AMT_DRAWINGS_ATM_CURRENT_min_mean is a duplicate of column prev_credit_AMT_DRAWINGS_ATM_CURRENT_min_median
column prev_credit_AMT_DRAWINGS_ATM_CURRENT_mean_mean is a duplicate of column prev_credit_AMT_DRAWINGS_ATM_CURRENT_mean_median
column prev_credit_AMT_DRAWINGS_OTHER_CURRENT_min_mean is a duplicate of column prev_credit_AMT_DRAWINGS_OTHER_CURRENT_min_median
c

In [16]:
feature_dict = {'NAME_CONTRACT_TYPE':'category_features', 'CODE_GENDER':'category_features', 'FLAG_OWN_CAR':'boolean_features', 'FLAG_OWN_REALTY':'boolean_features', 'CNT_CHILDREN':'count_features', 'AMT_INCOME_TOTAL':'amount_feautres', 'AMT_CREDIT':'amount_feautres', 'AMT_ANNUITY':'amount_feautres', 'AMT_GOODS_PRICE':'amount_feautres', 'NAME_TYPE_SUITE':'category_features', 'NAME_INCOME_TYPE':'category_features', 'NAME_EDUCATION_TYPE':'category_features', 'NAME_FAMILY_STATUS':'category_features', 'NAME_HOUSING_TYPE':'category_features', 'REGION_POPULATION_RELATIVE':'normal_features', 'DAYS_BIRTH':'amount_feautres', 'DAYS_EMPLOYED':'amount_feautres', 'DAYS_REGISTRATION':'amount_feautres', 'DAYS_ID_PUBLISH':'amount_feautres', 'OWN_CAR_AGE':'amount_feautres', 'FLAG_MOBIL':'boolean_features', 'FLAG_EMP_PHONE':'boolean_features', 'FLAG_WORK_PHONE':'boolean_features', 'FLAG_CONT_MOBILE':'boolean_features', 'FLAG_PHONE':'boolean_features', 'FLAG_EMAIL':'boolean_features', 'OCCUPATION_TYPE':'category_features', 'CNT_FAM_MEMBERS':'count_features', 'REGION_RATING_CLIENT':'category_features', 'REGION_RATING_CLIENT_W_CITY':'category_features', 'WEEKDAY_APPR_PROCESS_START':'category_features', 'HOUR_APPR_PROCESS_START':'category_features', 'REG_REGION_NOT_LIVE_REGION':'boolean_features', 'REG_REGION_NOT_WORK_REGION':'boolean_features', 'LIVE_REGION_NOT_WORK_REGION':'boolean_features', 'REG_CITY_NOT_LIVE_CITY':'boolean_features', 'REG_CITY_NOT_WORK_CITY':'boolean_features', 'LIVE_CITY_NOT_WORK_CITY':'boolean_features', 'ORGANIZATION_TYPE':'category_features', 'EXT_SOURCE_1':'normal_features', 'EXT_SOURCE_2':'normal_features', 'EXT_SOURCE_3':'normal_features', 'APARTMENTS_AVG':'normal_features', 'BASEMENTAREA_AVG':'normal_features', 'YEARS_BEGINEXPLUATATION_AVG':'normal_features', 'YEARS_BUILD_AVG':'normal_features', 'COMMONAREA_AVG':'normal_features', 'ELEVATORS_AVG':'normal_features', 'ENTRANCES_AVG':'normal_features', 'FLOORSMAX_AVG':'normal_features', 'FLOORSMIN_AVG':'normal_features', 'LANDAREA_AVG':'normal_features', 'LIVINGAPARTMENTS_AVG':'normal_features', 'LIVINGAREA_AVG':'normal_features', 'NONLIVINGAPARTMENTS_AVG':'normal_features', 'NONLIVINGAREA_AVG':'normal_features', 'APARTMENTS_MODE':'normal_features', 'BASEMENTAREA_MODE':'normal_features', 'YEARS_BEGINEXPLUATATION_MODE':'normal_features', 'YEARS_BUILD_MODE':'normal_features', 'COMMONAREA_MODE':'normal_features', 'ELEVATORS_MODE':'normal_features', 'ENTRANCES_MODE':'normal_features', 'FLOORSMAX_MODE':'normal_features', 'FLOORSMIN_MODE':'normal_features', 'LANDAREA_MODE':'normal_features', 'LIVINGAPARTMENTS_MODE':'normal_features', 'LIVINGAREA_MODE':'normal_features', 'NONLIVINGAPARTMENTS_MODE':'normal_features', 'NONLIVINGAREA_MODE':'normal_features', 'APARTMENTS_MEDI':'normal_features', 'BASEMENTAREA_MEDI':'normal_features', 'YEARS_BEGINEXPLUATATION_MEDI':'normal_features', 'YEARS_BUILD_MEDI':'normal_features', 'COMMONAREA_MEDI':'normal_features', 'ELEVATORS_MEDI':'normal_features', 'ENTRANCES_MEDI':'normal_features', 'FLOORSMAX_MEDI':'normal_features', 'FLOORSMIN_MEDI':'normal_features', 'LANDAREA_MEDI':'normal_features', 'LIVINGAPARTMENTS_MEDI':'normal_features', 'LIVINGAREA_MEDI':'normal_features', 'NONLIVINGAPARTMENTS_MEDI':'normal_features', 'NONLIVINGAREA_MEDI':'normal_features', 'FONDKAPREMONT_MODE':'category_features', 'HOUSETYPE_MODE':'category_features', 'TOTALAREA_MODE':'normal_features', 'WALLSMATERIAL_MODE':'category_features', 'EMERGENCYSTATE_MODE':'category_features', 'OBS_30_CNT_SOCIAL_CIRCLE':'count_features', 'DEF_30_CNT_SOCIAL_CIRCLE':'count_features', 'OBS_60_CNT_SOCIAL_CIRCLE':'count_features', 'DEF_60_CNT_SOCIAL_CIRCLE':'count_features', 'DAYS_LAST_PHONE_CHANGE':'count_features', 'FLAG_DOCUMENT_2':'boolean_features', 'FLAG_DOCUMENT_3':'boolean_features', 'FLAG_DOCUMENT_4':'boolean_features', 'FLAG_DOCUMENT_5':'boolean_features', 'FLAG_DOCUMENT_6':'boolean_features', 'FLAG_DOCUMENT_7':'boolean_features', 'FLAG_DOCUMENT_8':'boolean_features', 'FLAG_DOCUMENT_9':'boolean_features', 'FLAG_DOCUMENT_10':'boolean_features', 'FLAG_DOCUMENT_11':'boolean_features', 'FLAG_DOCUMENT_12':'boolean_features', 'FLAG_DOCUMENT_13':'boolean_features', 'FLAG_DOCUMENT_14':'boolean_features', 'FLAG_DOCUMENT_15':'boolean_features', 'FLAG_DOCUMENT_16':'boolean_features', 'FLAG_DOCUMENT_17':'boolean_features', 'FLAG_DOCUMENT_18':'boolean_features', 'FLAG_DOCUMENT_19':'boolean_features', 'FLAG_DOCUMENT_20':'boolean_features', 'FLAG_DOCUMENT_21':'boolean_features', 'AMT_REQ_CREDIT_BUREAU_HOUR':'count_features', 'AMT_REQ_CREDIT_BUREAU_DAY':'count_features', 'AMT_REQ_CREDIT_BUREAU_WEEK':'count_features', 'AMT_REQ_CREDIT_BUREAU_MON':'count_features', 'AMT_REQ_CREDIT_BUREAU_QRT':'count_features', 'AMT_REQ_CREDIT_BUREAU_YEAR':'count_features',
                'bureau_bureau_cnt':'count_features', 'bureau_CREDIT_DAY_OVERDUE_sum':'amount_feautres', 'bureau_DAYS_CREDIT_ENDDATE_sum':'amount_feautres', 'bureau_DAYS_ENDDATE_FACT_sum':'amount_feautres', 'bureau_DAYS_CREDIT_sum':'amount_feautres', 'bureau_DAYS_CREDIT_UPDATE_sum':'amount_feautres', 'bureau_CNT_CREDIT_PROLONG_sum':'amount_feautres', 'bureau_AMT_CREDIT_SUM_sum':'amount_feautres', 'bureau_AMT_ANNUITY_sum':'amount_feautres', 'bureau_AMT_CREDIT_SUM_DEBT_sum':'amount_feautres', 'bureau_AMT_CREDIT_SUM_LIMIT_sum':'amount_feautres', 'bureau_AMT_CREDIT_MAX_OVERDUE_sum':'amount_feautres', 'bureau_MONTHS_BALANCE_max_sum':'amount_feautres', 'bureau_MONTHS_BALANCE_cnt_sum':'amount_feautres', 'bureau_STATUS_cntd_sum':'amount_feautres', 'bureau_AMT_CREDIT_MAX_OVERDUE_max':'amount_feautres', 'bureau_CNT_CREDIT_PROLONG_max':'amount_feautres', 'bureau_DAYS_CREDIT_max':'amount_feautres', 'bureau_MONTHS_BALANCE_max_max':'amount_feautres', 'bureau_STATUS_cntd_max':'amount_feautres', 'bureau_CREDIT_CURRENCY_cnt':'amount_feautres', 'bureau_CREDIT_ACTIVE_cnt':'amount_feautres', 'bureau_CREDIT_TYPE_cnt':'amount_feautres', 'bureau_STATUS_set_cnt':'amount_feautres', 'bureau_latest_STATUS_cnt':'amount_feautres', 'bureau_CREDIT_CURRENCY_set':'category_features', 'bureau_CREDIT_ACTIVE_set':'category_features', 'bureau_CREDIT_TYPE_set':'category_features', 'bureau_latest_STATUS_set':'category_features', 'bureau_STATUS_set':'category_features',
                'cash_MONTHS_BALANCE_min':'count_features', 'cash_MONTHS_BALANCE_max':'count_features', 'cash_MONTHS_BALANCE_cnt':'count_features', 'cash_CNT_INSTALMENT_min':'count_features', 'cash_CNT_INSTALMENT_max':'count_features', 'cash_CNT_INSTALMENT_set':'category_features', 'cash_CNT_INSTALMENT_FUTURE_min':'count_features', 'cash_CNT_INSTALMENT_FUTURE_max':'count_features', 'cash_NAME_CONTRACT_STATUS_cntd':'count_features', 'cash_NAME_CONTRACT_STATUS_set':'category_features', 'cash_NAME_CONTRACT_STATUS_latest':'category_features', 'cash_SK_DPD_max':'count_features', 'cash_SK_DPD_cnt0':'count_features', 'cash_SK_DPD_DEF_max':'count_features', 'cash_SK_DPD_DEF_cnt0':'count_features', 'cash_SK_DPD_diff_max':'count_features', 'cash_SK_DPD_diff_cnt0':'count_features',
               'prev_PREV_cnt':'count_features', 'prev_AMT_DOWN_PAYMENT_cnt_negative':'count_features', 'prev_AMT_DOWN_PAYMENT_cnt_positive':'count_features', 'prev_HOUR_APPR_PROCESS_START_median':'count_features', 'prev_FLAG_LAST_APPL_PER_CONTRACT_1':'boolean_features', 'prev_NFLAG_LAST_APPL_IN_DAY_1':'boolean_features', 'prev_NFLAG_INSURED_ON_APPROVAL':'boolean_features', 'prev_NAME_CONTRACT_STATUS_Approved':'boolean_features', 'prev_NAME_CONTRACT_STATUS_Canceled':'boolean_features', 'prev_NAME_CONTRACT_STATUS_Refused':'boolean_features', 'prev_NAME_CONTRACT_STATUS_Unused_offer':'boolean_features', 'prev_NAME_PAYMENT_TYPE_Cash_through_the_bank':'boolean_features', 'prev_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer':'boolean_features', 'prev_NAME_PAYMENT_TYPE_Non-cash_from_your_account':'boolean_features', 'prev_NAME_PAYMENT_TYPE_XNA':'boolean_features', 'prev_CODE_REJECT_REASON_CLIENT':'boolean_features', 'prev_CODE_REJECT_REASON_HC':'boolean_features', 'prev_CODE_REJECT_REASON_LIMIT':'boolean_features', 'prev_CODE_REJECT_REASON_SCO':'boolean_features', 'prev_CODE_REJECT_REASON_SCOFR':'boolean_features', 'prev_CODE_REJECT_REASON_SYSTEM':'boolean_features', 'prev_CODE_REJECT_REASON_VERIF':'boolean_features', 'prev_CODE_REJECT_REASON_XAP':'boolean_features', 'prev_CODE_REJECT_REASON_XNA':'boolean_features', 'prev_NAME_TYPE_SUITE_Children':'boolean_features', 'prev_NAME_TYPE_SUITE_Family':'boolean_features', 'prev_NAME_TYPE_SUITE_Group_of_people':'boolean_features', 'prev_NAME_TYPE_SUITE_Other_A':'boolean_features', 'prev_NAME_TYPE_SUITE_Other_B':'boolean_features', 'prev_NAME_TYPE_SUITE_Spouse__partner':'boolean_features', 'prev_NAME_TYPE_SUITE_Unaccompanied':'boolean_features', 'prev_NAME_CLIENT_TYPE_New':'boolean_features', 'prev_NAME_CLIENT_TYPE_Refreshed':'boolean_features', 'prev_NAME_CLIENT_TYPE_Repeater':'boolean_features', 'prev_NAME_CLIENT_TYPE_XNA':'boolean_features', 'prev_NAME_PORTFOLIO_Cards':'boolean_features', 'prev_NAME_PORTFOLIO_Cars':'boolean_features', 'prev_NAME_PORTFOLIO_Cash':'boolean_features', 'prev_NAME_PORTFOLIO_POS':'boolean_features', 'prev_NAME_PORTFOLIO_XNA':'boolean_features', 'prev_NAME_PRODUCT_TYPE_XNA':'boolean_features', 'prev_NAME_PRODUCT_TYPE_walk-in':'boolean_features', 'prev_NAME_PRODUCT_TYPE_x-sell':'boolean_features', 'prev_CHANNEL_TYPE_AP+_(Cash_loan)':'boolean_features', 'prev_CHANNEL_TYPE_Car_dealer':'boolean_features', 'prev_CHANNEL_TYPE_Channel_of_corporate_sales':'boolean_features', 'prev_CHANNEL_TYPE_Contact_center':'boolean_features', 'prev_CHANNEL_TYPE_Country-wide':'boolean_features', 'prev_CHANNEL_TYPE_Credit_and_cash_offices':'boolean_features', 'prev_CHANNEL_TYPE_Regional_/_Local':'boolean_features', 'prev_CHANNEL_TYPE_Stone':'boolean_features', 'prev_NAME_SELLER_INDUSTRY_Auto_technology':'boolean_features', 'prev_NAME_SELLER_INDUSTRY_Clothing':'boolean_features', 'prev_NAME_SELLER_INDUSTRY_Connectivity':'boolean_features', 'prev_NAME_SELLER_INDUSTRY_Construction':'boolean_features', 'prev_NAME_SELLER_INDUSTRY_Consumer_electronics':'boolean_features', 'prev_NAME_SELLER_INDUSTRY_Furniture':'boolean_features', 'prev_NAME_SELLER_INDUSTRY_Industry':'boolean_features', 'prev_NAME_SELLER_INDUSTRY_Jewelry':'boolean_features', 'prev_NAME_SELLER_INDUSTRY_MLM_partners':'boolean_features', 'prev_NAME_SELLER_INDUSTRY_Tourism':'boolean_features', 'prev_NAME_SELLER_INDUSTRY_XNA':'boolean_features', 'prev_NAME_YIELD_GROUP_XNA':'boolean_features', 'prev_NAME_YIELD_GROUP_high':'boolean_features', 'prev_NAME_YIELD_GROUP_low_action':'boolean_features', 'prev_NAME_YIELD_GROUP_low_normal':'boolean_features', 'prev_NAME_YIELD_GROUP_middle':'boolean_features', 'prev_PRODUCT_COMBINATION_Card_Street':'boolean_features', 'prev_PRODUCT_COMBINATION_Card_X-Sell':'boolean_features', 'prev_PRODUCT_COMBINATION_Cash':'boolean_features', 'prev_PRODUCT_COMBINATION_Cash_Street__high':'boolean_features', 'prev_PRODUCT_COMBINATION_Cash_Street__low':'boolean_features', 'prev_PRODUCT_COMBINATION_Cash_Street__middle':'boolean_features', 'prev_PRODUCT_COMBINATION_Cash_X-Sell__high':'boolean_features', 'prev_PRODUCT_COMBINATION_Cash_X-Sell__low':'boolean_features', 'prev_PRODUCT_COMBINATION_Cash_X-Sell__middle':'boolean_features', 'prev_PRODUCT_COMBINATION_POS_household_with_interest':'boolean_features', 'prev_PRODUCT_COMBINATION_POS_household_without_interest':'boolean_features', 'prev_PRODUCT_COMBINATION_POS_industry_with_interest':'boolean_features', 'prev_PRODUCT_COMBINATION_POS_industry_without_interest':'boolean_features', 'prev_PRODUCT_COMBINATION_POS_mobile_with_interest':'boolean_features', 'prev_PRODUCT_COMBINATION_POS_mobile_without_interest':'boolean_features', 'prev_PRODUCT_COMBINATION_POS_other_with_interest':'boolean_features', 'prev_PRODUCT_COMBINATION_POS_others_without_interest':'boolean_features', 'prev_RATE_DOWN_PAYMENT_min':'normal_features', 'prev_RATE_DOWN_PAYMENT_max':'normal_features', 'prev_RATE_DOWN_PAYMENT_mean':'normal_features', 'prev_RATE_DOWN_PAYMENT_median':'normal_features', 'prev_RATE_DOWN_PAYMENT_latest':'normal_features', 'prev_RATE_INTEREST_PRIMARY_min':'normal_features', 'prev_RATE_INTEREST_PRIMARY_max':'normal_features', 'prev_RATE_INTEREST_PRIMARY_mean':'normal_features', 'prev_RATE_INTEREST_PRIMARY_median':'normal_features', 'prev_RATE_INTEREST_PRIMARY_latest':'normal_features', 'prev_RATE_INTEREST_PRIVILEGED_min':'normal_features', 'prev_RATE_INTEREST_PRIVILEGED_max':'normal_features', 'prev_RATE_INTEREST_PRIVILEGED_mean':'normal_features', 'prev_RATE_INTEREST_PRIVILEGED_median':'normal_features', 'prev_RATE_INTEREST_PRIVILEGED_latest':'normal_features', 'prev_DAYS_DECISION_min':'amount_feautres', 'prev_DAYS_DECISION_max':'amount_feautres', 'prev_DAYS_DECISION_mean':'amount_feautres', 'prev_DAYS_DECISION_median':'amount_feautres', 'prev_DAYS_DECISION_latest':'amount_feautres', 'prev_SELLERPLACE_AREA_min':'amount_feautres', 'prev_SELLERPLACE_AREA_max':'amount_feautres', 'prev_SELLERPLACE_AREA_mean':'amount_feautres', 'prev_SELLERPLACE_AREA_median':'amount_feautres', 'prev_SELLERPLACE_AREA_latest':'amount_feautres', 'prev_DAYS_FIRST_DRAWING_min':'amount_feautres', 'prev_DAYS_FIRST_DRAWING_max':'amount_feautres', 'prev_DAYS_FIRST_DRAWING_mean':'amount_feautres', 'prev_DAYS_FIRST_DRAWING_median':'amount_feautres', 'prev_DAYS_FIRST_DRAWING_latest':'amount_feautres', 'prev_DAYS_FIRST_DUE_min':'amount_feautres', 'prev_DAYS_FIRST_DUE_max':'amount_feautres', 'prev_DAYS_FIRST_DUE_mean':'amount_feautres', 'prev_DAYS_FIRST_DUE_median':'amount_feautres', 'prev_DAYS_FIRST_DUE_latest':'amount_feautres', 'prev_DAYS_LAST_DUE_1ST_VERSION_min':'amount_feautres', 'prev_DAYS_LAST_DUE_1ST_VERSION_max':'amount_feautres', 'prev_DAYS_LAST_DUE_1ST_VERSION_mean':'amount_feautres', 'prev_DAYS_LAST_DUE_1ST_VERSION_median':'amount_feautres', 'prev_DAYS_LAST_DUE_1ST_VERSION_latest':'amount_feautres', 'prev_DAYS_LAST_DUE_min':'amount_feautres', 'prev_DAYS_LAST_DUE_max':'amount_feautres', 'prev_DAYS_LAST_DUE_mean':'amount_feautres', 'prev_DAYS_LAST_DUE_median':'amount_feautres', 'prev_DAYS_LAST_DUE_latest':'amount_feautres', 'prev_DAYS_TERMINATION_min':'amount_feautres', 'prev_DAYS_TERMINATION_max':'amount_feautres', 'prev_DAYS_TERMINATION_mean':'amount_feautres', 'prev_DAYS_TERMINATION_median':'amount_feautres', 'prev_DAYS_TERMINATION_latest':'amount_feautres', 'prev_cash_MONTHS_BALANCE_min_min':'count_features', 'prev_cash_MONTHS_BALANCE_min_max':'count_features', 'prev_cash_MONTHS_BALANCE_min_mean':'count_features', 'prev_cash_MONTHS_BALANCE_min_median':'count_features', 'prev_cash_MONTHS_BALANCE_min_latest':'count_features', 'prev_cash_MONTHS_BALANCE_max_min':'count_features', 'prev_cash_MONTHS_BALANCE_max_max':'count_features', 'prev_cash_MONTHS_BALANCE_max_mean':'count_features', 'prev_cash_MONTHS_BALANCE_max_median':'count_features', 'prev_cash_MONTHS_BALANCE_max_latest':'count_features', 'prev_cash_MONTHS_BALANCE_cnt_min':'count_features', 'prev_cash_MONTHS_BALANCE_cnt_max':'count_features', 'prev_cash_MONTHS_BALANCE_cnt_mean':'count_features', 'prev_cash_MONTHS_BALANCE_cnt_median':'count_features', 'prev_cash_MONTHS_BALANCE_cnt_latest':'count_features', 'prev_cash_CNT_INSTALMENT_min_min':'count_features', 'prev_cash_CNT_INSTALMENT_min_max':'count_features', 'prev_cash_CNT_INSTALMENT_min_mean':'count_features', 'prev_cash_CNT_INSTALMENT_min_median':'count_features', 'prev_cash_CNT_INSTALMENT_min_latest':'count_features', 'prev_cash_CNT_INSTALMENT_max_min':'count_features', 'prev_cash_CNT_INSTALMENT_max_max':'count_features', 'prev_cash_CNT_INSTALMENT_max_mean':'count_features', 'prev_cash_CNT_INSTALMENT_max_median':'count_features', 'prev_cash_CNT_INSTALMENT_max_latest':'count_features', 'prev_cash_CNT_INSTALMENT_FUTURE_min_min':'count_features', 'prev_cash_CNT_INSTALMENT_FUTURE_min_max':'count_features', 'prev_cash_CNT_INSTALMENT_FUTURE_min_mean':'count_features', 'prev_cash_CNT_INSTALMENT_FUTURE_min_median':'count_features', 'prev_cash_CNT_INSTALMENT_FUTURE_min_latest':'count_features', 'prev_cash_CNT_INSTALMENT_FUTURE_max_min':'count_features', 'prev_cash_CNT_INSTALMENT_FUTURE_max_max':'count_features', 'prev_cash_CNT_INSTALMENT_FUTURE_max_mean':'count_features', 'prev_cash_CNT_INSTALMENT_FUTURE_max_median':'count_features', 'prev_cash_CNT_INSTALMENT_FUTURE_max_latest':'count_features', 'prev_cash_NAME_CONTRACT_STATUS_cntd_min':'count_features', 'prev_cash_NAME_CONTRACT_STATUS_cntd_max':'count_features', 'prev_cash_NAME_CONTRACT_STATUS_cntd_mean':'count_features', 'prev_cash_NAME_CONTRACT_STATUS_cntd_median':'count_features', 'prev_cash_NAME_CONTRACT_STATUS_cntd_latest':'count_features', 'prev_cash_SK_DPD_max_min':'count_features', 'prev_cash_SK_DPD_max_max':'count_features', 'prev_cash_SK_DPD_max_mean':'count_features', 'prev_cash_SK_DPD_max_median':'count_features', 'prev_cash_SK_DPD_max_latest':'count_features', 'prev_cash_SK_DPD_cnt0_min':'count_features', 'prev_cash_SK_DPD_cnt0_max':'count_features', 'prev_cash_SK_DPD_cnt0_mean':'count_features', 'prev_cash_SK_DPD_cnt0_median':'count_features', 'prev_cash_SK_DPD_cnt0_latest':'count_features', 'prev_cash_SK_DPD_DEF_max_min':'count_features', 'prev_cash_SK_DPD_DEF_max_max':'count_features', 'prev_cash_SK_DPD_DEF_max_mean':'count_features', 'prev_cash_SK_DPD_DEF_max_median':'count_features', 'prev_cash_SK_DPD_DEF_max_latest':'count_features', 'prev_cash_SK_DPD_DEF_cnt0_min':'count_features', 'prev_cash_SK_DPD_DEF_cnt0_max':'count_features', 'prev_cash_SK_DPD_DEF_cnt0_mean':'count_features', 'prev_cash_SK_DPD_DEF_cnt0_median':'count_features', 'prev_cash_SK_DPD_DEF_cnt0_latest':'count_features', 'prev_cash_SK_DPD_diff_max_min':'count_features', 'prev_cash_SK_DPD_diff_max_max':'count_features', 'prev_cash_SK_DPD_diff_max_mean':'count_features', 'prev_cash_SK_DPD_diff_max_median':'count_features', 'prev_cash_SK_DPD_diff_max_latest':'count_features', 'prev_cash_SK_DPD_diff_cnt0_min':'count_features', 'prev_cash_SK_DPD_diff_cnt0_max':'count_features', 'prev_cash_SK_DPD_diff_cnt0_mean':'count_features', 'prev_cash_SK_DPD_diff_cnt0_median':'count_features', 'prev_cash_SK_DPD_diff_cnt0_latest':'count_features', 'prev_credit_MONTHS_BALANCE_min_min':'count_features', 'prev_credit_MONTHS_BALANCE_min_max':'count_features', 'prev_credit_MONTHS_BALANCE_min_mean':'count_features', 'prev_credit_MONTHS_BALANCE_min_median':'count_features', 'prev_credit_MONTHS_BALANCE_min_latest':'count_features', 'prev_credit_MONTHS_BALANCE_max_min':'count_features', 'prev_credit_MONTHS_BALANCE_max_max':'count_features', 'prev_credit_MONTHS_BALANCE_max_mean':'count_features', 'prev_credit_MONTHS_BALANCE_max_median':'count_features', 'prev_credit_MONTHS_BALANCE_max_latest':'count_features', 'prev_credit_MONTHS_BALANCE_count_min':'count_features', 'prev_credit_MONTHS_BALANCE_count_max':'count_features', 'prev_credit_MONTHS_BALANCE_count_mean':'count_features', 'prev_credit_MONTHS_BALANCE_count_median':'count_features', 'prev_credit_MONTHS_BALANCE_count_latest':'count_features', 'prev_credit_AMT_BALANCE_min_min':'amount_feautres', 'prev_credit_AMT_BALANCE_min_max':'amount_feautres', 'prev_credit_AMT_BALANCE_min_mean':'amount_feautres', 'prev_credit_AMT_BALANCE_min_median':'amount_feautres', 'prev_credit_AMT_BALANCE_min_latest':'amount_feautres', 'prev_credit_AMT_BALANCE_max_min':'amount_feautres', 'prev_credit_AMT_BALANCE_max_max':'amount_feautres', 'prev_credit_AMT_BALANCE_max_mean':'amount_feautres', 'prev_credit_AMT_BALANCE_max_median':'amount_feautres', 'prev_credit_AMT_BALANCE_max_latest':'amount_feautres', 'prev_credit_AMT_BALANCE_sum_min':'amount_feautres', 'prev_credit_AMT_BALANCE_sum_max':'amount_feautres', 'prev_credit_AMT_BALANCE_sum_mean':'amount_feautres', 'prev_credit_AMT_BALANCE_sum_median':'amount_feautres', 'prev_credit_AMT_BALANCE_sum_latest':'amount_feautres', 'prev_credit_AMT_BALANCE_mean_min':'amount_feautres', 'prev_credit_AMT_BALANCE_mean_max':'amount_feautres', 'prev_credit_AMT_BALANCE_mean_mean':'amount_feautres', 'prev_credit_AMT_BALANCE_mean_median':'amount_feautres', 'prev_credit_AMT_BALANCE_mean_latest':'amount_feautres', 'prev_credit_AMT_BALANCE_size_min':'count_features', 'prev_credit_AMT_BALANCE_size_max':'count_features', 'prev_credit_AMT_BALANCE_size_mean':'count_features', 'prev_credit_AMT_BALANCE_size_median':'count_features', 'prev_credit_AMT_BALANCE_size_latest':'count_features', 'prev_credit_AMT_CREDIT_LIMIT_ACTUAL_min_min':'amount_feautres', 'prev_credit_AMT_CREDIT_LIMIT_ACTUAL_min_max':'amount_feautres', 'prev_credit_AMT_CREDIT_LIMIT_ACTUAL_min_mean':'amount_feautres', 'prev_credit_AMT_CREDIT_LIMIT_ACTUAL_min_median':'amount_feautres', 'prev_credit_AMT_CREDIT_LIMIT_ACTUAL_min_latest':'amount_feautres', 'prev_credit_AMT_CREDIT_LIMIT_ACTUAL_max_min':'amount_feautres', 'prev_credit_AMT_CREDIT_LIMIT_ACTUAL_max_max':'amount_feautres', 'prev_credit_AMT_CREDIT_LIMIT_ACTUAL_max_mean':'amount_feautres', 'prev_credit_AMT_CREDIT_LIMIT_ACTUAL_max_median':'amount_feautres', 'prev_credit_AMT_CREDIT_LIMIT_ACTUAL_max_latest':'amount_feautres', 'prev_credit_AMT_DRAWINGS_ATM_CURRENT_min_min':'amount_feautres', 'prev_credit_AMT_DRAWINGS_ATM_CURRENT_min_max':'amount_feautres', 'prev_credit_AMT_DRAWINGS_ATM_CURRENT_min_mean':'amount_feautres', 'prev_credit_AMT_DRAWINGS_ATM_CURRENT_min_median':'amount_feautres', 'prev_credit_AMT_DRAWINGS_ATM_CURRENT_min_latest':'amount_feautres', 'prev_credit_AMT_DRAWINGS_ATM_CURRENT_size_min':'count_features', 'prev_credit_AMT_DRAWINGS_ATM_CURRENT_size_max':'count_features', 'prev_credit_AMT_DRAWINGS_ATM_CURRENT_size_mean':'count_features', 'prev_credit_AMT_DRAWINGS_ATM_CURRENT_size_median':'count_features', 'prev_credit_AMT_DRAWINGS_ATM_CURRENT_size_latest':'count_features', 'prev_credit_AMT_DRAWINGS_ATM_CURRENT_mean_min':'amount_feautres', 'prev_credit_AMT_DRAWINGS_ATM_CURRENT_mean_max':'amount_feautres', 'prev_credit_AMT_DRAWINGS_ATM_CURRENT_mean_mean':'amount_feautres', 'prev_credit_AMT_DRAWINGS_ATM_CURRENT_mean_median':'amount_feautres', 'prev_credit_AMT_DRAWINGS_ATM_CURRENT_mean_latest':'amount_feautres', 'prev_credit_AMT_DRAWINGS_ATM_CURRENT_sum_min':'amount_feautres', 'prev_credit_AMT_DRAWINGS_ATM_CURRENT_sum_max':'amount_feautres', 'prev_credit_AMT_DRAWINGS_ATM_CURRENT_sum_mean':'amount_feautres', 'prev_credit_AMT_DRAWINGS_ATM_CURRENT_sum_median':'amount_feautres', 'prev_credit_AMT_DRAWINGS_ATM_CURRENT_sum_latest':'amount_feautres', 'prev_credit_AMT_DRAWINGS_ATM_CURRENT_max_min':'amount_feautres', 'prev_credit_AMT_DRAWINGS_ATM_CURRENT_max_max':'amount_feautres', 'prev_credit_AMT_DRAWINGS_ATM_CURRENT_max_mean':'amount_feautres', 'prev_credit_AMT_DRAWINGS_ATM_CURRENT_max_median':'amount_feautres', 'prev_credit_AMT_DRAWINGS_ATM_CURRENT_max_latest':'amount_feautres', 'prev_credit_AMT_DRAWINGS_CURRENT_min_min':'amount_feautres', 'prev_credit_AMT_DRAWINGS_CURRENT_min_max':'amount_feautres', 'prev_credit_AMT_DRAWINGS_CURRENT_min_mean':'amount_feautres', 'prev_credit_AMT_DRAWINGS_CURRENT_min_median':'amount_feautres', 'prev_credit_AMT_DRAWINGS_CURRENT_min_latest':'amount_feautres', 'prev_credit_AMT_DRAWINGS_CURRENT_size_min':'count_features', 'prev_credit_AMT_DRAWINGS_CURRENT_size_max':'count_features', 'prev_credit_AMT_DRAWINGS_CURRENT_size_mean':'count_features', 'prev_credit_AMT_DRAWINGS_CURRENT_size_median':'count_features', 'prev_credit_AMT_DRAWINGS_CURRENT_size_latest':'count_features', 'prev_credit_AMT_DRAWINGS_CURRENT_mean_min':'amount_feautres', 'prev_credit_AMT_DRAWINGS_CURRENT_mean_max':'amount_feautres', 'prev_credit_AMT_DRAWINGS_CURRENT_mean_mean':'amount_feautres', 'prev_credit_AMT_DRAWINGS_CURRENT_mean_median':'amount_feautres', 'prev_credit_AMT_DRAWINGS_CURRENT_mean_latest':'amount_feautres', 'prev_credit_AMT_DRAWINGS_CURRENT_sum_min':'amount_feautres', 'prev_credit_AMT_DRAWINGS_CURRENT_sum_max':'amount_feautres', 'prev_credit_AMT_DRAWINGS_CURRENT_sum_mean':'amount_feautres', 'prev_credit_AMT_DRAWINGS_CURRENT_sum_median':'amount_feautres', 'prev_credit_AMT_DRAWINGS_CURRENT_sum_latest':'amount_feautres', 'prev_credit_AMT_DRAWINGS_CURRENT_max_min':'amount_feautres', 'prev_credit_AMT_DRAWINGS_CURRENT_max_max':'amount_feautres', 'prev_credit_AMT_DRAWINGS_CURRENT_max_mean':'amount_feautres', 'prev_credit_AMT_DRAWINGS_CURRENT_max_median':'amount_feautres', 'prev_credit_AMT_DRAWINGS_CURRENT_max_latest':'amount_feautres', 'prev_credit_AMT_DRAWINGS_OTHER_CURRENT_min_min':'amount_feautres', 'prev_credit_AMT_DRAWINGS_OTHER_CURRENT_min_max':'amount_feautres', 'prev_credit_AMT_DRAWINGS_OTHER_CURRENT_min_mean':'amount_feautres', 'prev_credit_AMT_DRAWINGS_OTHER_CURRENT_min_median':'amount_feautres', 'prev_credit_AMT_DRAWINGS_OTHER_CURRENT_min_latest':'amount_feautres', 'prev_credit_AMT_DRAWINGS_OTHER_CURRENT_size_min':'count_features', 'prev_credit_AMT_DRAWINGS_OTHER_CURRENT_size_max':'count_features', 'prev_credit_AMT_DRAWINGS_OTHER_CURRENT_size_mean':'count_features', 'prev_credit_AMT_DRAWINGS_OTHER_CURRENT_size_median':'count_features', 'prev_credit_AMT_DRAWINGS_OTHER_CURRENT_size_latest':'count_features', 'prev_credit_AMT_DRAWINGS_OTHER_CURRENT_mean_min':'amount_feautres', 'prev_credit_AMT_DRAWINGS_OTHER_CURRENT_mean_max':'amount_feautres', 'prev_credit_AMT_DRAWINGS_OTHER_CURRENT_mean_mean':'amount_feautres', 'prev_credit_AMT_DRAWINGS_OTHER_CURRENT_mean_median':'amount_feautres', 'prev_credit_AMT_DRAWINGS_OTHER_CURRENT_mean_latest':'amount_feautres', 'prev_credit_AMT_DRAWINGS_OTHER_CURRENT_sum_min':'amount_feautres', 'prev_credit_AMT_DRAWINGS_OTHER_CURRENT_sum_max':'amount_feautres', 'prev_credit_AMT_DRAWINGS_OTHER_CURRENT_sum_mean':'amount_feautres', 'prev_credit_AMT_DRAWINGS_OTHER_CURRENT_sum_median':'amount_feautres', 'prev_credit_AMT_DRAWINGS_OTHER_CURRENT_sum_latest':'amount_feautres', 'prev_credit_AMT_DRAWINGS_OTHER_CURRENT_max_min':'amount_feautres', 'prev_credit_AMT_DRAWINGS_OTHER_CURRENT_max_max':'amount_feautres', 'prev_credit_AMT_DRAWINGS_OTHER_CURRENT_max_mean':'amount_feautres', 'prev_credit_AMT_DRAWINGS_OTHER_CURRENT_max_median':'amount_feautres', 'prev_credit_AMT_DRAWINGS_OTHER_CURRENT_max_latest':'amount_feautres', 'prev_credit_AMT_DRAWINGS_POS_CURRENT_min_min':'amount_feautres', 'prev_credit_AMT_DRAWINGS_POS_CURRENT_min_max':'amount_feautres', 'prev_credit_AMT_DRAWINGS_POS_CURRENT_min_mean':'amount_feautres', 'prev_credit_AMT_DRAWINGS_POS_CURRENT_min_median':'amount_feautres', 'prev_credit_AMT_DRAWINGS_POS_CURRENT_min_latest':'amount_feautres', 'prev_credit_AMT_DRAWINGS_POS_CURRENT_size_min':'count_features', 'prev_credit_AMT_DRAWINGS_POS_CURRENT_size_max':'count_features', 'prev_credit_AMT_DRAWINGS_POS_CURRENT_size_mean':'count_features', 'prev_credit_AMT_DRAWINGS_POS_CURRENT_size_median':'count_features', 'prev_credit_AMT_DRAWINGS_POS_CURRENT_size_latest':'count_features', 'prev_credit_AMT_DRAWINGS_POS_CURRENT_mean_min':'amount_feautres', 'prev_credit_AMT_DRAWINGS_POS_CURRENT_mean_max':'amount_feautres', 'prev_credit_AMT_DRAWINGS_POS_CURRENT_mean_mean':'amount_feautres', 'prev_credit_AMT_DRAWINGS_POS_CURRENT_mean_median':'amount_feautres', 'prev_credit_AMT_DRAWINGS_POS_CURRENT_mean_latest':'amount_feautres', 'prev_credit_AMT_DRAWINGS_POS_CURRENT_sum_min':'amount_feautres', 'prev_credit_AMT_DRAWINGS_POS_CURRENT_sum_max':'amount_feautres', 'prev_credit_AMT_DRAWINGS_POS_CURRENT_sum_mean':'amount_feautres', 'prev_credit_AMT_DRAWINGS_POS_CURRENT_sum_median':'amount_feautres', 'prev_credit_AMT_DRAWINGS_POS_CURRENT_sum_latest':'amount_feautres', 'prev_credit_AMT_DRAWINGS_POS_CURRENT_max_min':'amount_feautres', 'prev_credit_AMT_DRAWINGS_POS_CURRENT_max_max':'amount_feautres', 'prev_credit_AMT_DRAWINGS_POS_CURRENT_max_mean':'amount_feautres', 'prev_credit_AMT_DRAWINGS_POS_CURRENT_max_median':'amount_feautres', 'prev_credit_AMT_DRAWINGS_POS_CURRENT_max_latest':'amount_feautres', 'prev_credit_AMT_INST_MIN_REGULARITY_min_min':'amount_feautres', 'prev_credit_AMT_INST_MIN_REGULARITY_min_max':'amount_feautres', 'prev_credit_AMT_INST_MIN_REGULARITY_min_mean':'amount_feautres', 'prev_credit_AMT_INST_MIN_REGULARITY_min_median':'amount_feautres', 'prev_credit_AMT_INST_MIN_REGULARITY_min_latest':'amount_feautres', 'prev_credit_AMT_INST_MIN_REGULARITY_max_min':'amount_feautres', 'prev_credit_AMT_INST_MIN_REGULARITY_max_max':'amount_feautres', 'prev_credit_AMT_INST_MIN_REGULARITY_max_mean':'amount_feautres', 'prev_credit_AMT_INST_MIN_REGULARITY_max_median':'amount_feautres', 'prev_credit_AMT_INST_MIN_REGULARITY_max_latest':'amount_feautres', 'prev_credit_AMT_INST_MIN_REGULARITY_cntd_min':'count_features', 'prev_credit_AMT_INST_MIN_REGULARITY_cntd_max':'count_features', 'prev_credit_AMT_INST_MIN_REGULARITY_cntd_mean':'count_features', 'prev_credit_AMT_INST_MIN_REGULARITY_cntd_median':'count_features', 'prev_credit_AMT_INST_MIN_REGULARITY_cntd_latest':'count_features', 'prev_credit_AMT_PAYMENT_CURRENT_sum_min':'amount_feautres', 'prev_credit_AMT_PAYMENT_CURRENT_sum_max':'amount_feautres', 'prev_credit_AMT_PAYMENT_CURRENT_sum_mean':'amount_feautres', 'prev_credit_AMT_PAYMENT_CURRENT_sum_median':'amount_feautres', 'prev_credit_AMT_PAYMENT_CURRENT_sum_latest':'amount_feautres', 'prev_credit_AMT_PAYMENT_CURRENT_max_min':'amount_feautres', 'prev_credit_AMT_PAYMENT_CURRENT_max_max':'amount_feautres', 'prev_credit_AMT_PAYMENT_CURRENT_max_mean':'amount_feautres', 'prev_credit_AMT_PAYMENT_CURRENT_max_median':'amount_feautres', 'prev_credit_AMT_PAYMENT_CURRENT_max_latest':'amount_feautres', 'prev_credit_AMT_PAYMENT_CURRENT_cntd_min':'count_features', 'prev_credit_AMT_PAYMENT_CURRENT_cntd_max':'count_features', 'prev_credit_AMT_PAYMENT_CURRENT_cntd_mean':'count_features', 'prev_credit_AMT_PAYMENT_CURRENT_cntd_median':'count_features', 'prev_credit_AMT_PAYMENT_CURRENT_cntd_latest':'count_features', 'prev_credit_AMT_PAYMENT_TOTAL_CURRENT_sum_min':'amount_feautres', 'prev_credit_AMT_PAYMENT_TOTAL_CURRENT_sum_max':'amount_feautres', 'prev_credit_AMT_PAYMENT_TOTAL_CURRENT_sum_mean':'amount_feautres', 'prev_credit_AMT_PAYMENT_TOTAL_CURRENT_sum_median':'amount_feautres', 'prev_credit_AMT_PAYMENT_TOTAL_CURRENT_sum_latest':'amount_feautres', 'prev_credit_AMT_PAYMENT_TOTAL_CURRENT_max_min':'amount_feautres', 'prev_credit_AMT_PAYMENT_TOTAL_CURRENT_max_max':'amount_feautres', 'prev_credit_AMT_PAYMENT_TOTAL_CURRENT_max_mean':'amount_feautres', 'prev_credit_AMT_PAYMENT_TOTAL_CURRENT_max_median':'amount_feautres', 'prev_credit_AMT_PAYMENT_TOTAL_CURRENT_max_latest':'amount_feautres', 'prev_credit_AMT_PAYMENT_TOTAL_CURRENT_cntd_min':'count_features', 'prev_credit_AMT_PAYMENT_TOTAL_CURRENT_cntd_max':'count_features', 'prev_credit_AMT_PAYMENT_TOTAL_CURRENT_cntd_mean':'count_features', 'prev_credit_AMT_PAYMENT_TOTAL_CURRENT_cntd_median':'count_features', 'prev_credit_AMT_PAYMENT_TOTAL_CURRENT_cntd_latest':'count_features', 'prev_credit_AMT_RECEIVABLE_PRINCIPAL_sum_min':'amount_feautres', 'prev_credit_AMT_RECEIVABLE_PRINCIPAL_sum_max':'amount_feautres', 'prev_credit_AMT_RECEIVABLE_PRINCIPAL_sum_mean':'amount_feautres', 'prev_credit_AMT_RECEIVABLE_PRINCIPAL_sum_median':'amount_feautres', 'prev_credit_AMT_RECEIVABLE_PRINCIPAL_sum_latest':'amount_feautres', 'prev_credit_AMT_RECEIVABLE_PRINCIPAL_max_min':'amount_feautres', 'prev_credit_AMT_RECEIVABLE_PRINCIPAL_max_max':'amount_feautres', 'prev_credit_AMT_RECEIVABLE_PRINCIPAL_max_mean':'amount_feautres', 'prev_credit_AMT_RECEIVABLE_PRINCIPAL_max_median':'amount_feautres', 'prev_credit_AMT_RECEIVABLE_PRINCIPAL_max_latest':'amount_feautres', 'prev_credit_AMT_RECEIVABLE_PRINCIPAL_cntd_min':'count_features', 'prev_credit_AMT_RECEIVABLE_PRINCIPAL_cntd_max':'count_features', 'prev_credit_AMT_RECEIVABLE_PRINCIPAL_cntd_mean':'count_features', 'prev_credit_AMT_RECEIVABLE_PRINCIPAL_cntd_median':'count_features', 'prev_credit_AMT_RECEIVABLE_PRINCIPAL_cntd_latest':'count_features', 'prev_credit_AMT_RECIVABLE_sum_min':'amount_feautres', 'prev_credit_AMT_RECIVABLE_sum_max':'amount_feautres', 'prev_credit_AMT_RECIVABLE_sum_mean':'amount_feautres', 'prev_credit_AMT_RECIVABLE_sum_median':'amount_feautres', 'prev_credit_AMT_RECIVABLE_sum_latest':'amount_feautres', 'prev_credit_AMT_RECIVABLE_max_min':'amount_feautres', 'prev_credit_AMT_RECIVABLE_max_max':'amount_feautres', 'prev_credit_AMT_RECIVABLE_max_mean':'amount_feautres', 'prev_credit_AMT_RECIVABLE_max_median':'amount_feautres', 'prev_credit_AMT_RECIVABLE_max_latest':'amount_feautres', 'prev_credit_AMT_RECIVABLE_cntd_min':'count_features', 'prev_credit_AMT_RECIVABLE_cntd_max':'count_features', 'prev_credit_AMT_RECIVABLE_cntd_mean':'count_features', 'prev_credit_AMT_RECIVABLE_cntd_median':'count_features', 'prev_credit_AMT_RECIVABLE_cntd_latest':'count_features', 'prev_credit_AMT_TOTAL_RECEIVABLE_sum_min':'amount_feautres', 'prev_credit_AMT_TOTAL_RECEIVABLE_sum_max':'amount_feautres', 'prev_credit_AMT_TOTAL_RECEIVABLE_sum_mean':'amount_feautres', 'prev_credit_AMT_TOTAL_RECEIVABLE_sum_median':'amount_feautres', 'prev_credit_AMT_TOTAL_RECEIVABLE_sum_latest':'amount_feautres', 'prev_credit_AMT_TOTAL_RECEIVABLE_max_min':'amount_feautres', 'prev_credit_AMT_TOTAL_RECEIVABLE_max_max':'amount_feautres', 'prev_credit_AMT_TOTAL_RECEIVABLE_max_mean':'amount_feautres', 'prev_credit_AMT_TOTAL_RECEIVABLE_max_median':'amount_feautres', 'prev_credit_AMT_TOTAL_RECEIVABLE_max_latest':'amount_feautres', 'prev_credit_AMT_TOTAL_RECEIVABLE_cntd_min':'count_features', 'prev_credit_AMT_TOTAL_RECEIVABLE_cntd_max':'count_features', 'prev_credit_AMT_TOTAL_RECEIVABLE_cntd_mean':'count_features', 'prev_credit_AMT_TOTAL_RECEIVABLE_cntd_median':'count_features', 'prev_credit_AMT_TOTAL_RECEIVABLE_cntd_latest':'count_features', 'prev_credit_CNT_DRAWINGS_ATM_CURRENT_sum_min':'count_features', 'prev_credit_CNT_DRAWINGS_ATM_CURRENT_sum_max':'count_features', 'prev_credit_CNT_DRAWINGS_ATM_CURRENT_sum_mean':'count_features', 'prev_credit_CNT_DRAWINGS_ATM_CURRENT_sum_median':'count_features', 'prev_credit_CNT_DRAWINGS_ATM_CURRENT_sum_latest':'count_features', 'prev_credit_CNT_DRAWINGS_ATM_CURRENT_max_min':'count_features', 'prev_credit_CNT_DRAWINGS_ATM_CURRENT_max_max':'count_features', 'prev_credit_CNT_DRAWINGS_ATM_CURRENT_max_mean':'count_features', 'prev_credit_CNT_DRAWINGS_ATM_CURRENT_max_median':'count_features', 'prev_credit_CNT_DRAWINGS_ATM_CURRENT_max_latest':'count_features', 'prev_credit_CNT_DRAWINGS_ATM_CURRENT_mean_min':'count_features', 'prev_credit_CNT_DRAWINGS_ATM_CURRENT_mean_max':'count_features', 'prev_credit_CNT_DRAWINGS_ATM_CURRENT_mean_mean':'count_features', 'prev_credit_CNT_DRAWINGS_ATM_CURRENT_mean_median':'count_features', 'prev_credit_CNT_DRAWINGS_ATM_CURRENT_mean_latest':'count_features', 'prev_credit_CNT_DRAWINGS_CURRENT_sum_min':'count_features', 'prev_credit_CNT_DRAWINGS_CURRENT_sum_max':'count_features', 'prev_credit_CNT_DRAWINGS_CURRENT_sum_mean':'count_features', 'prev_credit_CNT_DRAWINGS_CURRENT_sum_median':'count_features', 'prev_credit_CNT_DRAWINGS_CURRENT_sum_latest':'count_features', 'prev_credit_CNT_DRAWINGS_CURRENT_max_min':'count_features', 'prev_credit_CNT_DRAWINGS_CURRENT_max_max':'count_features', 'prev_credit_CNT_DRAWINGS_CURRENT_max_mean':'count_features', 'prev_credit_CNT_DRAWINGS_CURRENT_max_median':'count_features', 'prev_credit_CNT_DRAWINGS_CURRENT_max_latest':'count_features', 'prev_credit_CNT_DRAWINGS_CURRENT_mean_min':'count_features', 'prev_credit_CNT_DRAWINGS_CURRENT_mean_max':'count_features', 'prev_credit_CNT_DRAWINGS_CURRENT_mean_mean':'count_features', 'prev_credit_CNT_DRAWINGS_CURRENT_mean_median':'count_features', 'prev_credit_CNT_DRAWINGS_CURRENT_mean_latest':'count_features', 'prev_credit_CNT_DRAWINGS_OTHER_CURRENT_sum_min':'count_features', 'prev_credit_CNT_DRAWINGS_OTHER_CURRENT_sum_max':'count_features', 'prev_credit_CNT_DRAWINGS_OTHER_CURRENT_sum_mean':'count_features', 'prev_credit_CNT_DRAWINGS_OTHER_CURRENT_sum_median':'count_features', 'prev_credit_CNT_DRAWINGS_OTHER_CURRENT_sum_latest':'count_features', 'prev_credit_CNT_DRAWINGS_OTHER_CURRENT_max_min':'count_features', 'prev_credit_CNT_DRAWINGS_OTHER_CURRENT_max_max':'count_features', 'prev_credit_CNT_DRAWINGS_OTHER_CURRENT_max_mean':'count_features', 'prev_credit_CNT_DRAWINGS_OTHER_CURRENT_max_median':'count_features', 'prev_credit_CNT_DRAWINGS_OTHER_CURRENT_max_latest':'count_features', 'prev_credit_CNT_DRAWINGS_OTHER_CURRENT_mean_min':'count_features', 'prev_credit_CNT_DRAWINGS_OTHER_CURRENT_mean_max':'count_features', 'prev_credit_CNT_DRAWINGS_OTHER_CURRENT_mean_mean':'count_features', 'prev_credit_CNT_DRAWINGS_OTHER_CURRENT_mean_median':'count_features', 'prev_credit_CNT_DRAWINGS_OTHER_CURRENT_mean_latest':'count_features', 'prev_credit_CNT_DRAWINGS_POS_CURRENT_sum_min':'count_features', 'prev_credit_CNT_DRAWINGS_POS_CURRENT_sum_max':'count_features', 'prev_credit_CNT_DRAWINGS_POS_CURRENT_sum_mean':'count_features', 'prev_credit_CNT_DRAWINGS_POS_CURRENT_sum_median':'count_features', 'prev_credit_CNT_DRAWINGS_POS_CURRENT_sum_latest':'count_features', 'prev_credit_CNT_DRAWINGS_POS_CURRENT_max_min':'count_features', 'prev_credit_CNT_DRAWINGS_POS_CURRENT_max_max':'count_features', 'prev_credit_CNT_DRAWINGS_POS_CURRENT_max_mean':'count_features', 'prev_credit_CNT_DRAWINGS_POS_CURRENT_max_median':'count_features', 'prev_credit_CNT_DRAWINGS_POS_CURRENT_max_latest':'count_features', 'prev_credit_CNT_DRAWINGS_POS_CURRENT_mean_min':'count_features', 'prev_credit_CNT_DRAWINGS_POS_CURRENT_mean_max':'count_features', 'prev_credit_CNT_DRAWINGS_POS_CURRENT_mean_mean':'count_features', 'prev_credit_CNT_DRAWINGS_POS_CURRENT_mean_median':'count_features', 'prev_credit_CNT_DRAWINGS_POS_CURRENT_mean_latest':'count_features', 'prev_credit_CNT_INSTALMENT_MATURE_CUM_cntd_min':'count_features', 'prev_credit_CNT_INSTALMENT_MATURE_CUM_cntd_max':'count_features', 'prev_credit_CNT_INSTALMENT_MATURE_CUM_cntd_mean':'count_features', 'prev_credit_CNT_INSTALMENT_MATURE_CUM_cntd_median':'count_features', 'prev_credit_CNT_INSTALMENT_MATURE_CUM_cntd_latest':'count_features', 'prev_credit_SK_DPD_max_min':'count_features', 'prev_credit_SK_DPD_max_max':'count_features', 'prev_credit_SK_DPD_max_mean':'count_features', 'prev_credit_SK_DPD_max_median':'count_features', 'prev_credit_SK_DPD_max_latest':'count_features', 'prev_credit_SK_DPD_size_min':'count_features', 'prev_credit_SK_DPD_size_max':'count_features', 'prev_credit_SK_DPD_size_mean':'count_features', 'prev_credit_SK_DPD_size_median':'count_features', 'prev_credit_SK_DPD_size_latest':'count_features', 'prev_credit_SK_DPD_DEF_max_min':'count_features', 'prev_credit_SK_DPD_DEF_max_max':'count_features', 'prev_credit_SK_DPD_DEF_max_mean':'count_features', 'prev_credit_SK_DPD_DEF_max_median':'count_features', 'prev_credit_SK_DPD_DEF_max_latest':'count_features', 'prev_credit_SK_DPD_DEF_size_min':'count_features', 'prev_credit_SK_DPD_DEF_size_max':'count_features', 'prev_credit_SK_DPD_DEF_size_mean':'count_features', 'prev_credit_SK_DPD_DEF_size_median':'count_features', 'prev_credit_SK_DPD_DEF_size_latest':'count_features', 'prev_credit_SK_DPD_diff_max_min':'count_features', 'prev_credit_SK_DPD_diff_max_max':'count_features', 'prev_credit_SK_DPD_diff_max_mean':'count_features', 'prev_credit_SK_DPD_diff_max_median':'count_features', 'prev_credit_SK_DPD_diff_max_latest':'count_features', 'prev_credit_SK_DPD_diff_size_min':'count_features', 'prev_credit_SK_DPD_diff_size_max':'count_features', 'prev_credit_SK_DPD_diff_size_mean':'count_features', 'prev_credit_SK_DPD_diff_size_median':'count_features', 'prev_credit_SK_DPD_diff_size_latest':'count_features', 'prev_installment_NUM_INSTALMENT_VERSION_nunique_min':'count_features', 'prev_installment_NUM_INSTALMENT_VERSION_nunique_max':'count_features', 'prev_installment_NUM_INSTALMENT_VERSION_nunique_mean':'count_features', 'prev_installment_NUM_INSTALMENT_VERSION_nunique_median':'count_features', 'prev_installment_NUM_INSTALMENT_VERSION_nunique_latest':'count_features', 'prev_installment_NUM_INSTALMENT_NUMBER_min_min':'count_features', 'prev_installment_NUM_INSTALMENT_NUMBER_min_max':'count_features', 'prev_installment_NUM_INSTALMENT_NUMBER_min_mean':'count_features', 'prev_installment_NUM_INSTALMENT_NUMBER_min_median':'count_features', 'prev_installment_NUM_INSTALMENT_NUMBER_min_latest':'count_features', 'prev_installment_NUM_INSTALMENT_NUMBER_max_min':'count_features', 'prev_installment_NUM_INSTALMENT_NUMBER_max_max':'count_features', 'prev_installment_NUM_INSTALMENT_NUMBER_max_mean':'count_features', 'prev_installment_NUM_INSTALMENT_NUMBER_max_median':'count_features', 'prev_installment_NUM_INSTALMENT_NUMBER_max_latest':'count_features', 'prev_installment_NUM_INSTALMENT_NUMBER_count_min':'count_features', 'prev_installment_NUM_INSTALMENT_NUMBER_count_max':'count_features', 'prev_installment_NUM_INSTALMENT_NUMBER_count_mean':'count_features', 'prev_installment_NUM_INSTALMENT_NUMBER_count_median':'count_features', 'prev_installment_NUM_INSTALMENT_NUMBER_count_latest':'count_features', 'prev_installment_NUM_INSTALMENT_NUMBER_nunique_min':'count_features', 'prev_installment_NUM_INSTALMENT_NUMBER_nunique_max':'count_features', 'prev_installment_NUM_INSTALMENT_NUMBER_nunique_mean':'count_features', 'prev_installment_NUM_INSTALMENT_NUMBER_nunique_median':'count_features', 'prev_installment_NUM_INSTALMENT_NUMBER_nunique_latest':'count_features', 'prev_installment_DAYS_INSTALMENT_min_min':'count_features', 'prev_installment_DAYS_INSTALMENT_min_max':'count_features', 'prev_installment_DAYS_INSTALMENT_min_mean':'count_features', 'prev_installment_DAYS_INSTALMENT_min_median':'count_features', 'prev_installment_DAYS_INSTALMENT_min_latest':'count_features', 'prev_installment_DAYS_INSTALMENT_max_min':'count_features', 'prev_installment_DAYS_INSTALMENT_max_max':'count_features', 'prev_installment_DAYS_INSTALMENT_max_mean':'count_features', 'prev_installment_DAYS_INSTALMENT_max_median':'count_features', 'prev_installment_DAYS_INSTALMENT_max_latest':'count_features', 'prev_installment_DAYS_INSTALMENT_median_min':'count_features', 'prev_installment_DAYS_INSTALMENT_median_max':'count_features', 'prev_installment_DAYS_INSTALMENT_median_mean':'count_features', 'prev_installment_DAYS_INSTALMENT_median_median':'count_features', 'prev_installment_DAYS_INSTALMENT_median_latest':'count_features', 'prev_installment_DAYS_INSTALMENT_mean_min':'count_features', 'prev_installment_DAYS_INSTALMENT_mean_max':'count_features', 'prev_installment_DAYS_INSTALMENT_mean_mean':'count_features', 'prev_installment_DAYS_INSTALMENT_mean_median':'count_features', 'prev_installment_DAYS_INSTALMENT_mean_latest':'count_features', 'prev_installment_DAYS_INSTALMENT_range_min':'count_features', 'prev_installment_DAYS_INSTALMENT_range_max':'count_features', 'prev_installment_DAYS_INSTALMENT_range_mean':'count_features', 'prev_installment_DAYS_INSTALMENT_range_median':'count_features', 'prev_installment_DAYS_INSTALMENT_range_latest':'count_features', 'prev_installment_DAYS_ENTRY_PAYMENT_min_min':'count_features', 'prev_installment_DAYS_ENTRY_PAYMENT_min_max':'count_features', 'prev_installment_DAYS_ENTRY_PAYMENT_min_mean':'count_features', 'prev_installment_DAYS_ENTRY_PAYMENT_min_median':'count_features', 'prev_installment_DAYS_ENTRY_PAYMENT_min_latest':'count_features', 'prev_installment_DAYS_ENTRY_PAYMENT_max_min':'count_features', 'prev_installment_DAYS_ENTRY_PAYMENT_max_max':'count_features', 'prev_installment_DAYS_ENTRY_PAYMENT_max_mean':'count_features', 'prev_installment_DAYS_ENTRY_PAYMENT_max_median':'count_features', 'prev_installment_DAYS_ENTRY_PAYMENT_max_latest':'count_features', 'prev_installment_DAYS_ENTRY_PAYMENT_median_min':'count_features', 'prev_installment_DAYS_ENTRY_PAYMENT_median_max':'count_features', 'prev_installment_DAYS_ENTRY_PAYMENT_median_mean':'count_features', 'prev_installment_DAYS_ENTRY_PAYMENT_median_median':'count_features', 'prev_installment_DAYS_ENTRY_PAYMENT_median_latest':'count_features', 'prev_installment_DAYS_ENTRY_PAYMENT_mean_min':'count_features', 'prev_installment_DAYS_ENTRY_PAYMENT_mean_max':'count_features', 'prev_installment_DAYS_ENTRY_PAYMENT_mean_mean':'count_features', 'prev_installment_DAYS_ENTRY_PAYMENT_mean_median':'count_features', 'prev_installment_DAYS_ENTRY_PAYMENT_mean_latest':'count_features', 'prev_installment_DAYS_ENTRY_PAYMENT_range_min':'count_features', 'prev_installment_DAYS_ENTRY_PAYMENT_range_max':'count_features', 'prev_installment_DAYS_ENTRY_PAYMENT_range_mean':'count_features', 'prev_installment_DAYS_ENTRY_PAYMENT_range_median':'count_features', 'prev_installment_DAYS_ENTRY_PAYMENT_range_latest':'count_features', 'prev_installment_DPD_min_min':'count_features', 'prev_installment_DPD_min_max':'count_features', 'prev_installment_DPD_min_mean':'count_features', 'prev_installment_DPD_min_median':'count_features', 'prev_installment_DPD_min_latest':'count_features', 'prev_installment_DPD_max_min':'count_features', 'prev_installment_DPD_max_max':'count_features', 'prev_installment_DPD_max_mean':'count_features', 'prev_installment_DPD_max_median':'count_features', 'prev_installment_DPD_max_latest':'count_features', 'prev_installment_DPD_median_min':'count_features', 'prev_installment_DPD_median_max':'count_features', 'prev_installment_DPD_median_mean':'count_features', 'prev_installment_DPD_median_median':'count_features', 'prev_installment_DPD_median_latest':'count_features', 'prev_installment_DPD_mean_min':'count_features', 'prev_installment_DPD_mean_max':'count_features', 'prev_installment_DPD_mean_mean':'count_features', 'prev_installment_DPD_mean_median':'count_features', 'prev_installment_DPD_mean_latest':'count_features', 'prev_installment_DPD_sum_min':'count_features', 'prev_installment_DPD_sum_max':'count_features', 'prev_installment_DPD_sum_mean':'count_features', 'prev_installment_DPD_sum_median':'count_features', 'prev_installment_DPD_sum_latest':'count_features', 'prev_installment_DPD_std_min':'count_features', 'prev_installment_DPD_std_max':'count_features', 'prev_installment_DPD_std_mean':'count_features', 'prev_installment_DPD_std_median':'count_features', 'prev_installment_DPD_std_latest':'count_features', 'prev_installment_AMT_INSTALMENT_min_min':'amount_feautres', 'prev_installment_AMT_INSTALMENT_min_max':'amount_feautres', 'prev_installment_AMT_INSTALMENT_min_mean':'amount_feautres', 'prev_installment_AMT_INSTALMENT_min_median':'amount_feautres', 'prev_installment_AMT_INSTALMENT_min_latest':'amount_feautres', 'prev_installment_AMT_INSTALMENT_max_min':'amount_feautres', 'prev_installment_AMT_INSTALMENT_max_max':'amount_feautres', 'prev_installment_AMT_INSTALMENT_max_mean':'amount_feautres', 'prev_installment_AMT_INSTALMENT_max_median':'amount_feautres', 'prev_installment_AMT_INSTALMENT_max_latest':'amount_feautres', 'prev_installment_AMT_INSTALMENT_mean_min':'amount_feautres', 'prev_installment_AMT_INSTALMENT_mean_max':'amount_feautres', 'prev_installment_AMT_INSTALMENT_mean_mean':'amount_feautres', 'prev_installment_AMT_INSTALMENT_mean_median':'amount_feautres', 'prev_installment_AMT_INSTALMENT_mean_latest':'amount_feautres', 'prev_installment_AMT_INSTALMENT_sum_min':'amount_feautres', 'prev_installment_AMT_INSTALMENT_sum_max':'amount_feautres', 'prev_installment_AMT_INSTALMENT_sum_mean':'amount_feautres', 'prev_installment_AMT_INSTALMENT_sum_median':'amount_feautres', 'prev_installment_AMT_INSTALMENT_sum_latest':'amount_feautres', 'prev_installment_AMT_INSTALMENT_median_min':'amount_feautres', 'prev_installment_AMT_INSTALMENT_median_max':'amount_feautres', 'prev_installment_AMT_INSTALMENT_median_mean':'amount_feautres', 'prev_installment_AMT_INSTALMENT_median_median':'amount_feautres', 'prev_installment_AMT_INSTALMENT_median_latest':'amount_feautres', 'prev_installment_AMT_INSTALMENT_std_min':'amount_feautres', 'prev_installment_AMT_INSTALMENT_std_max':'amount_feautres', 'prev_installment_AMT_INSTALMENT_std_mean':'amount_feautres', 'prev_installment_AMT_INSTALMENT_std_median':'amount_feautres', 'prev_installment_AMT_INSTALMENT_std_latest':'amount_feautres', 'prev_installment_AMT_INSTALMENT_nunique_min':'count_features', 'prev_installment_AMT_INSTALMENT_nunique_max':'count_features', 'prev_installment_AMT_INSTALMENT_nunique_mean':'count_features', 'prev_installment_AMT_INSTALMENT_nunique_median':'count_features', 'prev_installment_AMT_INSTALMENT_nunique_latest':'count_features', 'prev_installment_AMT_PAYMENT_min_min':'amount_feautres', 'prev_installment_AMT_PAYMENT_min_max':'amount_feautres', 'prev_installment_AMT_PAYMENT_min_mean':'amount_feautres', 'prev_installment_AMT_PAYMENT_min_median':'amount_feautres', 'prev_installment_AMT_PAYMENT_min_latest':'amount_feautres', 'prev_installment_AMT_PAYMENT_max_min':'amount_feautres', 'prev_installment_AMT_PAYMENT_max_max':'amount_feautres', 'prev_installment_AMT_PAYMENT_max_mean':'amount_feautres', 'prev_installment_AMT_PAYMENT_max_median':'amount_feautres', 'prev_installment_AMT_PAYMENT_max_latest':'amount_feautres', 'prev_installment_AMT_PAYMENT_mean_min':'amount_feautres', 'prev_installment_AMT_PAYMENT_mean_max':'amount_feautres', 'prev_installment_AMT_PAYMENT_mean_mean':'amount_feautres', 'prev_installment_AMT_PAYMENT_mean_median':'amount_feautres', 'prev_installment_AMT_PAYMENT_mean_latest':'amount_feautres', 'prev_installment_AMT_PAYMENT_sum_min':'amount_feautres', 'prev_installment_AMT_PAYMENT_sum_max':'amount_feautres', 'prev_installment_AMT_PAYMENT_sum_mean':'amount_feautres', 'prev_installment_AMT_PAYMENT_sum_median':'amount_feautres', 'prev_installment_AMT_PAYMENT_sum_latest':'amount_feautres', 'prev_installment_AMT_PAYMENT_median_min':'amount_feautres', 'prev_installment_AMT_PAYMENT_median_max':'amount_feautres', 'prev_installment_AMT_PAYMENT_median_mean':'amount_feautres', 'prev_installment_AMT_PAYMENT_median_median':'amount_feautres', 'prev_installment_AMT_PAYMENT_median_latest':'amount_feautres', 'prev_installment_AMT_PAYMENT_std_min':'amount_feautres', 'prev_installment_AMT_PAYMENT_std_max':'amount_feautres', 'prev_installment_AMT_PAYMENT_std_mean':'amount_feautres', 'prev_installment_AMT_PAYMENT_std_median':'amount_feautres', 'prev_installment_AMT_PAYMENT_std_latest':'amount_feautres', 'prev_installment_AMT_PAYMENT_nunique_min':'count_features', 'prev_installment_AMT_PAYMENT_nunique_max':'count_features', 'prev_installment_AMT_PAYMENT_nunique_mean':'count_features', 'prev_installment_AMT_PAYMENT_nunique_median':'count_features', 'prev_installment_AMT_PAYMENT_nunique_latest':'count_features', 'prev_installment_AMT_DPD_min_min':'count_features', 'prev_installment_AMT_DPD_min_max':'count_features', 'prev_installment_AMT_DPD_min_mean':'count_features', 'prev_installment_AMT_DPD_min_median':'count_features', 'prev_installment_AMT_DPD_min_latest':'count_features', 'prev_installment_AMT_DPD_max_min':'count_features', 'prev_installment_AMT_DPD_max_max':'count_features', 'prev_installment_AMT_DPD_max_mean':'count_features', 'prev_installment_AMT_DPD_max_median':'count_features', 'prev_installment_AMT_DPD_max_latest':'count_features', 'prev_installment_AMT_DPD_mean_min':'count_features', 'prev_installment_AMT_DPD_mean_max':'count_features', 'prev_installment_AMT_DPD_mean_mean':'count_features', 'prev_installment_AMT_DPD_mean_median':'count_features', 'prev_installment_AMT_DPD_mean_latest':'count_features', 'prev_installment_AMT_DPD_sum_min':'count_features', 'prev_installment_AMT_DPD_sum_max':'count_features', 'prev_installment_AMT_DPD_sum_mean':'count_features', 'prev_installment_AMT_DPD_sum_median':'count_features', 'prev_installment_AMT_DPD_sum_latest':'count_features', 'prev_installment_AMT_DPD_median_min':'count_features', 'prev_installment_AMT_DPD_median_max':'count_features', 'prev_installment_AMT_DPD_median_mean':'count_features', 'prev_installment_AMT_DPD_median_median':'count_features', 'prev_installment_AMT_DPD_median_latest':'count_features', 'prev_installment_AMT_DPD_std_min':'count_features', 'prev_installment_AMT_DPD_std_max':'count_features', 'prev_installment_AMT_DPD_std_mean':'count_features', 'prev_installment_AMT_DPD_std_median':'count_features', 'prev_installment_AMT_DPD_std_latest':'count_features', 'prev_installment_AMT_DPD_nunique_min':'count_features', 'prev_installment_AMT_DPD_nunique_max':'count_features', 'prev_installment_AMT_DPD_nunique_mean':'count_features', 'prev_installment_AMT_DPD_nunique_median':'count_features', 'prev_installment_AMT_DPD_nunique_latest':'count_features', 'prev_WEEKDAY_APPR_PROCESS_START_collect_set':'category_features', 'prev_WEEKDAY_APPR_PROCESS_START_countd':'count_features', 'prev_WEEKDAY_APPR_PROCESS_START_latest':'category_features', 'prev_HOUR_APPR_PROCESS_START_collect_set':'category_features', 'prev_HOUR_APPR_PROCESS_START_countd':'count_features', 'prev_HOUR_APPR_PROCESS_START_latest':'category_features', 'prev_NAME_CASH_LOAN_PURPOSE_collect_set':'category_features', 'prev_NAME_CASH_LOAN_PURPOSE_countd':'count_features', 'prev_NAME_CASH_LOAN_PURPOSE_latest':'category_features', 'prev_NAME_GOODS_CATEGORY_collect_set':'category_features', 'prev_NAME_GOODS_CATEGORY_countd':'count_features', 'prev_NAME_GOODS_CATEGORY_latest':'category_features', 'prev_cash_CNT_INSTALMENT_set_collect_set':'category_features', 'prev_cash_CNT_INSTALMENT_set_countd':'count_features', 'prev_cash_CNT_INSTALMENT_set_latest':'category_features', 'prev_cash_NAME_CONTRACT_STATUS_set_collect_set':'category_features', 'prev_cash_NAME_CONTRACT_STATUS_set_countd':'count_features', 'prev_cash_NAME_CONTRACT_STATUS_set_latest':'category_features', 'prev_cash_NAME_CONTRACT_STATUS_latest_collect_set':'category_features', 'prev_cash_NAME_CONTRACT_STATUS_latest_countd':'count_features', 'prev_cash_NAME_CONTRACT_STATUS_latest_latest':'category_features', 'prev_credit_CNT_INSTALMENT_MATURE_CUM_set_collect_set':'category_features', 'prev_credit_CNT_INSTALMENT_MATURE_CUM_set_countd':'count_features', 'prev_credit_CNT_INSTALMENT_MATURE_CUM_set_latest':'category_features', 'prev_credit_CNT_INSTALMENT_MATURE_CUM_latest_collect_set':'category_features', 'prev_credit_CNT_INSTALMENT_MATURE_CUM_latest_countd':'count_features', 'prev_credit_CNT_INSTALMENT_MATURE_CUM_latest_latest':'category_features', 'prev_credit_NAME_CONTRACT_STATUS_cntd_collect_set':'category_features', 'prev_credit_NAME_CONTRACT_STATUS_cntd_countd':'count_features', 'prev_credit_NAME_CONTRACT_STATUS_cntd_latest':'category_features', 'prev_credit_NAME_CONTRACT_STATUS_set_collect_set':'category_features', 'prev_credit_NAME_CONTRACT_STATUS_set_countd':'count_features', 'prev_credit_NAME_CONTRACT_STATUS_set_latest':'category_features', 'prev_installment_NUM_INSTALMENT_VERSION_set_collect_set':'category_features', 'prev_installment_NUM_INSTALMENT_VERSION_set_countd':'count_features', 'prev_installment_NUM_INSTALMENT_VERSION_set_latest':'category_features', 'prev_installment_NUM_INSTALMENT_VERSION_latest_collect_set':'category_features', 'prev_installment_NUM_INSTALMENT_VERSION_latest_countd''prev_installment_NUM_INSTALMENT_VERSION_latest_latest':'category_features', 'prev_NAME_CONTRACT_STATUS_collect_set':'category_features', 'prev_NAME_CONTRACT_STATUS_countd':'count_features', 'prev_NAME_CONTRACT_STATUS_latest':'category_features', 'prev_NAME_PAYMENT_TYPE_collect_set':'category_features', 'prev_NAME_PAYMENT_TYPE_countd':'count_features', 'prev_NAME_PAYMENT_TYPE_latest':'category_features', 'prev_CODE_REJECT_REASON_collect_set':'category_features', 'prev_CODE_REJECT_REASON_countd':'count_features', 'prev_CODE_REJECT_REASON_latest':'category_features', 'prev_NAME_TYPE_SUITE_collect_set':'category_features', 'prev_NAME_TYPE_SUITE_countd':'count_features', 'prev_NAME_TYPE_SUITE_latest':'category_features', 'prev_NAME_CLIENT_TYPE_collect_set':'category_features', 'prev_NAME_CLIENT_TYPE_countd':'count_features', 'prev_NAME_CLIENT_TYPE_latest':'category_features', 'prev_NAME_PORTFOLIO_collect_set':'category_features', 'prev_NAME_PORTFOLIO_countd':'count_features', 'prev_NAME_PORTFOLIO_latest':'category_features', 'prev_NAME_PRODUCT_TYPE_collect_set':'category_features', 'prev_NAME_PRODUCT_TYPE_countd':'count_features', 'prev_NAME_PRODUCT_TYPE_latest':'category_features', 'prev_CHANNEL_TYPE_collect_set':'category_features', 'prev_CHANNEL_TYPE_countd':'count_features', 'prev_CHANNEL_TYPE_latest':'category_features', 'prev_NAME_SELLER_INDUSTRY_collect_set':'category_features', 'prev_NAME_SELLER_INDUSTRY_countd':'count_features', 'prev_NAME_SELLER_INDUSTRY_latest':'category_features', 'prev_NAME_YIELD_GROUP_collect_set':'category_features', 'prev_NAME_YIELD_GROUP_countd':'count_features', 'prev_NAME_YIELD_GROUP_latest':'category_features', 'prev_PRODUCT_COMBINATION_collect_set':'category_features', 'prev_PRODUCT_COMBINATION_countd':'count_features', 'prev_PRODUCT_COMBINATION_latest':'category_features', 'prev_AMT_ANNUITY_sum':'amount_feautres', 'prev_AMT_ANNUITY_mean':'amount_feautres', 'prev_AMT_ANNUITY_max':'amount_feautres', 'prev_AMT_ANNUITY_min':'amount_feautres', 'prev_AMT_APPLICATION_sum':'amount_feautres', 'prev_AMT_APPLICATION_mean':'amount_feautres', 'prev_AMT_APPLICATION_max':'amount_feautres', 'prev_AMT_APPLICATION_min':'amount_feautres', 'prev_AMT_CREDIT_sum':'amount_feautres', 'prev_AMT_CREDIT_mean':'amount_feautres', 'prev_AMT_CREDIT_max':'amount_feautres', 'prev_AMT_CREDIT_min':'amount_feautres', 'prev_AMT_GOODS_PRICE_sum':'amount_feautres', 'prev_AMT_GOODS_PRICE_mean':'amount_feautres', 'prev_AMT_GOODS_PRICE_max':'amount_feautres', 'prev_AMT_GOODS_PRICE_min':'amount_feautres', 'prev_AMT_DOWN_PAYMENT_sum':'amount_feautres', 'prev_AMT_DOWN_PAYMENT_mean':'amount_feautres', 'prev_AMT_DOWN_PAYMENT_max':'amount_feautres', 'prev_AMT_DOWN_PAYMENT_min':'amount_feautres',
               }

In [17]:
category_features = [f for f in feature_dict if feature_dict[f] == 'category_features' and f in train_s.columns ]
boolean_features = [f for f in feature_dict if feature_dict[f] == 'boolean_features'  and f in train_s.columns ]
amount_feautres = [f for f in feature_dict if feature_dict[f] == 'amount_feautres'  and f in train_s.columns ]
normal_features = [f for f in feature_dict if feature_dict[f] == 'normal_features'  and f in train_s.columns ]
count_features = [f for f in feature_dict if feature_dict[f] == 'count_features'  and f in train_s.columns ]
del train_s

In [18]:

minmax = MinMaxScaler().fit(pd.concat([train[amount_feautres+count_features],test[amount_feautres+count_features]],axis = 0).fillna(0))

train[amount_feautres+count_features] = pd.DataFrame(minmax.transform(train[amount_feautres+count_features].fillna(0)), columns = amount_feautres+count_features)
test[amount_feautres+count_features] = pd.DataFrame(minmax.transform(test[amount_feautres+count_features].fillna(0)), columns = amount_feautres+count_features)


In [19]:
# bin
bins = 5
for rate in normal_features+amount_feautres+count_features:
    print(rate)
    qcut = pd.qcut(train[rate],bins,retbins=True,duplicates='drop')[1]
    if len(qcut) <=2:
        continue
    qcut[0] = 0
    qcut[-1] = 2
    train[rate +'_bin'] = [rate + '_' + str(np.where(qcut>= r)[0][0]) if not np.isnan(r) else 'miss' for r in train[rate]]
    test[rate +'_bin'] =  [rate + '_' +  str(np.where(qcut>= r)[0][0]) if not np.isnan(r) else 'miss' for r in test[rate]]
    category_features.append(rate +'_bin')


REGION_POPULATION_RELATIVE
EXT_SOURCE_1
EXT_SOURCE_2
EXT_SOURCE_3
APARTMENTS_AVG
BASEMENTAREA_AVG
YEARS_BEGINEXPLUATATION_AVG
YEARS_BUILD_AVG
COMMONAREA_AVG
ELEVATORS_AVG
ENTRANCES_AVG
FLOORSMAX_AVG
FLOORSMIN_AVG
LANDAREA_AVG
LIVINGAPARTMENTS_AVG
LIVINGAREA_AVG
NONLIVINGAPARTMENTS_AVG
NONLIVINGAREA_AVG
APARTMENTS_MODE
BASEMENTAREA_MODE
YEARS_BEGINEXPLUATATION_MODE
YEARS_BUILD_MODE
COMMONAREA_MODE
ELEVATORS_MODE
ENTRANCES_MODE
FLOORSMAX_MODE
FLOORSMIN_MODE
LANDAREA_MODE
LIVINGAPARTMENTS_MODE
LIVINGAREA_MODE
NONLIVINGAPARTMENTS_MODE
NONLIVINGAREA_MODE
APARTMENTS_MEDI
BASEMENTAREA_MEDI
YEARS_BEGINEXPLUATATION_MEDI
YEARS_BUILD_MEDI
COMMONAREA_MEDI
ELEVATORS_MEDI
ENTRANCES_MEDI
FLOORSMAX_MEDI
FLOORSMIN_MEDI
LANDAREA_MEDI
LIVINGAPARTMENTS_MEDI
LIVINGAREA_MEDI
NONLIVINGAPARTMENTS_MEDI
NONLIVINGAREA_MEDI
TOTALAREA_MODE
prev_RATE_DOWN_PAYMENT_min
prev_RATE_DOWN_PAYMENT_max
prev_RATE_DOWN_PAYMENT_mean
prev_RATE_DOWN_PAYMENT_median
prev_RATE_DOWN_PAYMENT_latest
prev_RATE_INTEREST_PRIMARY_min
prev

prev_credit_AMT_PAYMENT_TOTAL_CURRENT_max_latest
prev_credit_AMT_RECEIVABLE_PRINCIPAL_sum_min
prev_credit_AMT_RECEIVABLE_PRINCIPAL_sum_max
prev_credit_AMT_RECEIVABLE_PRINCIPAL_sum_mean
prev_credit_AMT_RECEIVABLE_PRINCIPAL_sum_median
prev_credit_AMT_RECEIVABLE_PRINCIPAL_sum_latest
prev_credit_AMT_RECEIVABLE_PRINCIPAL_max_min
prev_credit_AMT_RECEIVABLE_PRINCIPAL_max_max
prev_credit_AMT_RECEIVABLE_PRINCIPAL_max_mean
prev_credit_AMT_RECEIVABLE_PRINCIPAL_max_median
prev_credit_AMT_RECEIVABLE_PRINCIPAL_max_latest
prev_credit_AMT_RECIVABLE_sum_min
prev_credit_AMT_RECIVABLE_sum_max
prev_credit_AMT_RECIVABLE_sum_mean
prev_credit_AMT_RECIVABLE_sum_median
prev_credit_AMT_RECIVABLE_sum_latest
prev_credit_AMT_RECIVABLE_max_min
prev_credit_AMT_RECIVABLE_max_max
prev_credit_AMT_RECIVABLE_max_mean
prev_credit_AMT_RECIVABLE_max_median
prev_credit_AMT_RECIVABLE_max_latest
prev_credit_AMT_TOTAL_RECEIVABLE_sum_min
prev_credit_AMT_TOTAL_RECEIVABLE_sum_max
prev_credit_AMT_TOTAL_RECEIVABLE_sum_mean
prev_cred

prev_credit_AMT_DRAWINGS_OTHER_CURRENT_size_max
prev_credit_AMT_DRAWINGS_OTHER_CURRENT_size_median
prev_credit_AMT_DRAWINGS_OTHER_CURRENT_size_latest
prev_credit_AMT_DRAWINGS_POS_CURRENT_size_min
prev_credit_AMT_DRAWINGS_POS_CURRENT_size_max
prev_credit_AMT_DRAWINGS_POS_CURRENT_size_mean
prev_credit_AMT_DRAWINGS_POS_CURRENT_size_median
prev_credit_AMT_DRAWINGS_POS_CURRENT_size_latest
prev_credit_AMT_INST_MIN_REGULARITY_cntd_min
prev_credit_AMT_INST_MIN_REGULARITY_cntd_max
prev_credit_AMT_INST_MIN_REGULARITY_cntd_median
prev_credit_AMT_INST_MIN_REGULARITY_cntd_latest
prev_credit_AMT_PAYMENT_CURRENT_cntd_min
prev_credit_AMT_PAYMENT_CURRENT_cntd_max
prev_credit_AMT_PAYMENT_CURRENT_cntd_mean
prev_credit_AMT_PAYMENT_CURRENT_cntd_median
prev_credit_AMT_PAYMENT_CURRENT_cntd_latest
prev_credit_AMT_PAYMENT_TOTAL_CURRENT_cntd_min
prev_credit_AMT_PAYMENT_TOTAL_CURRENT_cntd_max
prev_credit_AMT_PAYMENT_TOTAL_CURRENT_cntd_mean
prev_credit_AMT_PAYMENT_TOTAL_CURRENT_cntd_median
prev_credit_AMT_PAYMENT

prev_installment_DAYS_ENTRY_PAYMENT_mean_max
prev_installment_DAYS_ENTRY_PAYMENT_mean_mean
prev_installment_DAYS_ENTRY_PAYMENT_mean_median
prev_installment_DAYS_ENTRY_PAYMENT_mean_latest
prev_installment_DAYS_ENTRY_PAYMENT_range_min
prev_installment_DAYS_ENTRY_PAYMENT_range_max
prev_installment_DAYS_ENTRY_PAYMENT_range_mean
prev_installment_DAYS_ENTRY_PAYMENT_range_median
prev_installment_DAYS_ENTRY_PAYMENT_range_latest
prev_installment_DPD_min_min
prev_installment_DPD_min_max
prev_installment_DPD_min_mean
prev_installment_DPD_min_median
prev_installment_DPD_min_latest
prev_installment_DPD_max_min
prev_installment_DPD_max_max
prev_installment_DPD_max_mean
prev_installment_DPD_max_median
prev_installment_DPD_max_latest
prev_installment_DPD_median_min
prev_installment_DPD_median_max
prev_installment_DPD_median_mean
prev_installment_DPD_median_median
prev_installment_DPD_median_latest
prev_installment_DPD_mean_min
prev_installment_DPD_mean_max
prev_installment_DPD_mean_mean
prev_installme

In [20]:
train.to_csv('./data/rawdata/train_all.tsv',index = False, sep = '\t')
test.to_csv('./data/rawdata/test_all.tsv',index = False, sep = '\t')

In [22]:
# 'cash_CNT_INSTALMENT_set'

category_low_card = [ x for x in category_features if train[x].unique().shape[0] < 128]
category_high_card = [ x for x in category_features if train[x].unique().shape[0] >= 128]
  
# taget_encoder = ce.TargetEncoder(cols = ['prev_cash_CNT_INSTALMENT_set_latest'],min_samples_leaf = 100,smoothing=0.03)
# taget_encoder.fit(train[['prev_cash_CNT_INSTALMENT_set_latest']], train.TARGET)

In [ ]:
taget_encoder = ce.TargetEncoder(cols = category_low_card,min_samples_leaf = 100,smoothing=0.03)
taget_encoder.fit(train[category_low_card], train.TARGET)

onehot_encoder = ce.OneHotEncoder(cols = category_low_card)
onehot_encoder.fit(train[category_low_card])

X_train = pd.concat([
            train[normal_features+amount_feautres+count_features].fillna(0),
            taget_encoder.transform(train[category_low_card]),
        onehot_encoder.transform(train[category_low_card]),
        ], axis = 1)

X_test = pd.concat([
            test[normal_features+amount_feautres+count_features].fillna(0),
            taget_encoder.transform(test[category_low_card]),
        onehot_encoder.transform(test[category_low_card]),
        ], axis = 1)

TargetEncoder(cols=['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'ORGANIZATION_TYPE', ...LER_INDUSTRY_countd_bin', 'prev_NAME_YIELD_GROUP_countd_bin', 'prev_PRODUCT_COMBINATION_countd_bin'],
       drop_invariant=False, handle_unknown='impute', impute_missing=True,
       min_samples_leaf=100, return_df=True, smoothing=0.03, verbose=0)

In [74]:
X_train = train[normal_features+amount_feautres+count_features].fillna(0)
X_test = test[normal_features+amount_feautres+count_features].fillna(0)

for f in category_features + boolean_features:
    print(f)
    if train[f].unique().shape[0] > 128:
        continue
    taget_encoder = ce.TargetEncoder(cols = [f],min_samples_leaf = 100,smoothing=0.03)
    taget_encoder.fit(train[[f]], train.TARGET)
    onehot_encoder = ce.OneHotEncoder(cols = [f])
    onehot_encoder.fit(train[[f]])
    
    if f in category_features:
        ordinal_encoder = ce.OrdinalEncoder(cols = [f])
        ordinal_encoder.fit(train[[f]])
        X_train = pd.concat([
            X_train,
            ordinal_encoder.transform(train[[f]]).rename(columns = {f:f+'_ordinal'}),
        ], axis = 1)
        X_test = pd.concat([
            X_test,
            ordinal_encoder.transform(test[[f]]).rename(columns = {f:f+'_ordinal'}),
        ], axis = 1)
        
    X_train = pd.concat([
        X_train,
        taget_encoder.transform(train[[f]]).rename(columns = {f:f+'_target'}),
        onehot_encoder.transform(train[[f]]),
    ], axis = 1)
    
    X_test = pd.concat([
        X_test,
        taget_encoder.transform(test[[f]]).rename(columns = {f:f+'_target'}),
        onehot_encoder.transform(test[[f]]),
    ], axis = 1)
    


y_train = train.TARGET


NAME_CONTRACT_TYPE


TargetEncoder(cols=['NAME_CONTRACT_TYPE'], drop_invariant=False,
       handle_unknown='impute', impute_missing=True, min_samples_leaf=100,
       return_df=True, smoothing=0.03, verbose=0)

OneHotEncoder(cols=['NAME_CONTRACT_TYPE'], drop_invariant=False,
       handle_unknown='impute', impute_missing=True, return_df=True,
       verbose=0)

OrdinalEncoder(cols=['NAME_CONTRACT_TYPE'], drop_invariant=False,
        handle_unknown='impute', impute_missing=True,
        mapping=[{'col': 'NAME_CONTRACT_TYPE', 'mapping': [('Cash loans', 0), ('Revolving loans', 1)]}],
        return_df=True, verbose=0)

CODE_GENDER


...

TargetEncoder(cols=['prev_PRODUCT_COMBINATION_POS_other_with_interest'],
       drop_invariant=False, handle_unknown='impute', impute_missing=True,
       min_samples_leaf=100, return_df=True, smoothing=0.03, verbose=0)

OneHotEncoder(cols=['prev_PRODUCT_COMBINATION_POS_other_with_interest'],
       drop_invariant=False, handle_unknown='impute', impute_missing=True,
       return_df=True, verbose=0)

prev_PRODUCT_COMBINATION_POS_others_without_interest


TargetEncoder(cols=['prev_PRODUCT_COMBINATION_POS_others_without_interest'],
       drop_invariant=False, handle_unknown='impute', impute_missing=True,
       min_samples_leaf=100, return_df=True, smoothing=0.03, verbose=0)

OneHotEncoder(cols=['prev_PRODUCT_COMBINATION_POS_others_without_interest'],
       drop_invariant=False, handle_unknown='impute', impute_missing=True,
       return_df=True, verbose=0)

In [ ]:
X_train.shape
X_train.head()

(307511, 10980)

,REGION_POPULATION_RELATIVE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,...,prev_PRODUCT_COMBINATION_POS_other_with_interest_target,prev_PRODUCT_COMBINATION_POS_other_with_interest_0,prev_PRODUCT_COMBINATION_POS_other_with_interest_1,prev_PRODUCT_COMBINATION_POS_other_with_interest_2,prev_PRODUCT_COMBINATION_POS_other_with_interest_-1,prev_PRODUCT_COMBINATION_POS_others_without_interest_target,prev_PRODUCT_COMBINATION_POS_others_without_interest_0,prev_PRODUCT_COMBINATION_POS_others_without_interest_1,prev_PRODUCT_COMBINATION_POS_others_without_interest_2,prev_PRODUCT_COMBINATION_POS_others_without_interest_-1
0,0.018801,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,...,0.079613,1,0,0,0,0.081991,1,0,0,0
1,0.003541,0.311267,0.622246,0.000000,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,...,0.082068,0,1,0,0,0.081991,1,0,0,0
2,0.010032,0.000000,0.555912,0.729567,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,...,0.082068,0,1,0,0,0.081991,1,0,0,0
3,0.008019,0.000000,0.650442,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,...,0.082068,0,1,0,0,0.081991,1,0,0,0
4,0.028663,0.000000,0.322738,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,...,0.082068,0,1,0,0,0.081991,1,0,0,0


In [ ]:
X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

lr = LogisticRegression(C = 0.1, penalty = 'l1', random_state = 42)

# pipeline = Pipeline([
#     ('select', SelectKBest(chi2, k=300)),
#     ('clf', lr)])

lr.fit(X_train_cv, y_train_cv)
roc_auc_score(y_train_cv, lr.predict_proba(X_train_cv)[:,1]), roc_auc_score(y_test_cv, lr.predict_proba(X_test_cv)[:,1])

In [ ]:
lr = LogisticRegression(C = 0.1, penalty = 'l1', random_state = 42)
lr.fit(X_train, y_train)
roc_auc_score(y_train, lr.predict_proba(X_train)[:,1])

In [ ]:
sub = pd.concat([
    test.SK_ID_CURR,
    pd.DataFrame({'TARGET':lr.predict_proba(X_test)[:,1]})
], axis = 1)


In [ ]:
sub.head()
sub.to_csv('./data/submission/forth_20180605.csv', index = False)

In [ ]:
pd.concat([X_train,pd.DataFrame(y_train, columns = ['TARGET'])], axis = 1).to_csv('./data/input/train.csv', index = False)
# y_train.to_csv('./data/input/y_train.csv', index = False)
X_test.to_csv('./data/input/test.csv', index = False)


In [35]:
entropy_select = mutual_info_classif(X_train, y_train)
mutual_info_selected = X_train.columns[entropy_select >= sorted(entropy_select,reverse = True)[99]]

f_var = X_train.apply(np.std,axis = 0)
var_selected = X_train.columns[f_var >= f_var.sort_values(ascending=False)[99]]


len(set(mutual_info_selected.tolist() + var_selected.tolist()))

In [52]:
ext_tree = ExtraTreesClassifier(n_estimators=500,random_state=42)

SelectFromModel(ext_tree)

179